## **<p style="text-align: center;">GoProtect</p>**

Сервис  “Мой Чемпион” помогает спортивным школам фигурного катания, тренерам мониторить результаты своих подопечных и планировать дальнейшее развитие спортсменов.

**Цель:**

Создать модель, помогающую находить элементы, которые могут быть успешно исполнены спортсменом на соревновании. 
Сервис будет прогнозировать прогресс и возможное выполнение тех или иных элементов программы по истории предыдущих выступлений и выполнения элементов на соревнованиях.



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

## Изучим предоставленные данные

In [2]:
total_scores_df = pd.read_csv('data/total_scores.csv')

total_scores_df

,id,unit_id,tournament_id,base_score,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str
0,442027,304,4785,47.20,43.47,102.70,59.23,0.0,17,1,Короткая программа,x Надбавка за прыжки во второй половине програ...,2,293.74,2
1,442028,604,4785,45.40,46.71,101.19,54.48,0.0,18,2,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,1,294.75,1
2,442029,409,4785,44.00,46.82,99.20,52.38,0.0,6,3,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,4,285.57,4
3,442030,524,4785,46.20,44.86,91.84,47.98,-1.0,3,4,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,3,292.42,3
4,442031,412,4785,44.20,42.59,89.73,48.14,-1.0,15,5,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,6,265.34,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21296,462405,34979,7116,21.67,26.02,46.62,21.10,-0.5,6,20,Произвольная программа,q Прыжок приземлён в четверть x Надбавка за пр...,20,71.66,20
21297,462406,1995,7116,23.20,25.50,46.22,21.22,-0.5,7,21,Произвольная программа,q Прыжок приземлён в четверть e Явно неправиль...,21,71.27,21
21298,462407,34841,7116,21.83,25.00,45.46,20.96,-0.5,5,22,Произвольная программа,q Прыжок приземлён в четверть << Пониженный пр...,22,70.49,22
21299,462408,34980,7116,21.17,26.00,45.24,20.74,-1.5,2,23,Произвольная программа,q Прыжок приземлён в четверть < Недокрученный ...,24,68.22,24


In [3]:
total_scores_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21301 entries, 0 to 21300
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   21301 non-null  int64  
 1   unit_id              21301 non-null  int64  
 2   tournament_id        21301 non-null  int64  
 3   base_score           21301 non-null  float64
 4   components_score     21301 non-null  float64
 5   total_score          21301 non-null  float64
 6   elements_score       21301 non-null  float64
 7   decreasings_score    21301 non-null  float64
 8   starting_place       21301 non-null  int64  
 9   place                21301 non-null  int64  
 10  segment_name         21284 non-null  object 
 11  info                 20720 non-null  object 
 12  overall_place        21301 non-null  int64  
 13  overall_total_score  21284 non-null  float64
 14  overall_place_str    10814 non-null  object 
dtypes: float64(6), int64(6), object(3)
m

In [4]:
total_scores_df['overall_place_str'].unique()

array(['2', '1', '4', '3', '6', '7', '5', '9', '13', '10', '11', '17',
       '12', '14', nan, '15', '16', '8', '18', '19', '25', '24', 'NQD',
       '23', '21', '20', 'WD', '22', '26', '28', '27', '40', '29', '30',
       '32', '31', '35', '33', '36', '34', '39', 'FNR', '41', '38', '44',
       '45', '37', '42', '43', '46', '47', '48', '49', '50'], dtype=object)

In [5]:
total_scores_df['overall_place'].unique()

array([ 2,  1,  4,  3,  6,  7,  5,  9, 13, 10, 11, 17, 12, 14, 24, 15, 16,
        8, 18,  0, 19, 25, 23, 21, 20, 22, 26, 28, 27, 32, 40, 29, 30, 31,
       35, 33, 36, 34, 39, 38, 41, 44, 45, 37, 42, 43, 46, 48, 49, 47, 50])

In [6]:
total_scores_df['overall_place_str'] = pd.to_numeric(total_scores_df['overall_place_str'], errors='coerce')

# сравниваем, True - совпадают, не совпадают False
total_scores_df['is_equal'] = np.where(total_scores_df['overall_place'] == total_scores_df['overall_place_str'], True, False)

total_scores_df[['overall_place', 'overall_place_str', 'is_equal']]

,overall_place,overall_place_str,is_equal
0,2,2.0,True
1,1,1.0,True
2,4,4.0,True
3,3,3.0,True
4,6,6.0,True
...,...,...,...
21296,20,20.0,True
21297,21,21.0,True
21298,22,22.0,True
21299,24,24.0,True


In [7]:
mismatched_rows = total_scores_df[total_scores_df['is_equal'] == False]

# Отображаем строки, где значения не совпадают
mismatched_rows[['overall_place', 'overall_place_str', 'is_equal']]

,overall_place,overall_place_str,is_equal
15,24,NaN,False
35,1,NaN,False
43,17,NaN,False
52,0,NaN,False
72,7,NaN,False
...,...,...,...
21216,2,NaN,False
21217,9,NaN,False
21230,0,NaN,False
21270,9,NaN,False


Я не нашла в описание данных конкретные обозначения в стобце (overall_place_str: комментарии, пояснения). Посчитала,что эти данные не несут важность,эту колонку удалим. 

In [8]:
total_scores_df = total_scores_df.drop(['is_equal', 'overall_place_str'], axis=1)

In [9]:
total_scores_df.isnull().sum()

id                       0
unit_id                  0
tournament_id            0
base_score               0
components_score         0
total_score              0
elements_score           0
decreasings_score        0
starting_place           0
place                    0
segment_name            17
info                   581
overall_place            0
overall_total_score     17
dtype: int64

In [10]:
duplicates = total_scores_df.duplicated()
print(f"Количество дубликатов: {duplicates.sum()}")

Количество дубликатов: 0


In [11]:
total_scores_df['base_score'].describe()

count    21301.000000
mean        27.809781
std         16.337113
min          0.000000
25%         15.560000
50%         24.490000
75%         35.110000
max         94.420000
Name: base_score, dtype: float64

Стандартное отклонение значительное, указывает на большой разброс в данных. Наличие нулевых значений критично для разработки модели, поскольку нулевые значения могут исказить анализ и прогнозы.

**Рекомендация!**

Сделать поле base_score обязательным для заполнения, чтобы избежать пропусков и ошибок в данных.

In [12]:
total_scores_df['total_score'].describe()

count    21301.000000
mean        60.410021
std         33.598123
min          0.000000
25%         35.880000
50%         52.630000
75%         75.720000
max        201.650000
Name: total_score, dtype: float64

In [13]:
zero_total_score = total_scores_df[total_scores_df['total_score'] == 0]

zero_total_score

,id,unit_id,tournament_id,base_score,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score
801,69255,282,69,0.0,0.0,0.0,0.0,0.0,0,0,Произвольная программа,NaN,0,0.0
1491,102234,282,69,0.0,0.0,0.0,0.0,0.0,0,0,Произвольная программа,NaN,0,0.0
2960,151675,282,72,0.0,0.0,0.0,0.0,0.0,0,0,Короткая программа,NaN,0,0.0
5461,272735,282,3,0.0,0.0,0.0,0.0,0.0,0,0,Короткая программа,NaN,0,0.0
9773,58109,282,84,0.0,0.0,0.0,0.0,0.0,0,0,Короткая программа,NaN,0,0.0
9993,294512,282,79,0.0,0.0,0.0,0.0,0.0,0,0,Короткая программа,NaN,0,0.0


Удалим строки с нулевыми значениями в total_score, так как нулевые значения присутствуют во всех остальных колонках. Такие строки не несут полезной информации для анализа и могут исказить статистические расчеты.

In [14]:
total_scores_df = total_scores_df[total_scores_df['total_score'] != 0]

In [15]:
total_scores_df['total_score'].describe()

count    21295.000000
mean        60.427042
std         33.587548
min          1.800000
25%         35.900000
50%         52.640000
75%         75.720000
max        201.650000
Name: total_score, dtype: float64

In [16]:
total_scores_df

,id,unit_id,tournament_id,base_score,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score
0,442027,304,4785,47.20,43.47,102.70,59.23,0.0,17,1,Короткая программа,x Надбавка за прыжки во второй половине програ...,2,293.74
1,442028,604,4785,45.40,46.71,101.19,54.48,0.0,18,2,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,1,294.75
2,442029,409,4785,44.00,46.82,99.20,52.38,0.0,6,3,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,4,285.57
3,442030,524,4785,46.20,44.86,91.84,47.98,-1.0,3,4,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,3,292.42
4,442031,412,4785,44.20,42.59,89.73,48.14,-1.0,15,5,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,6,265.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21296,462405,34979,7116,21.67,26.02,46.62,21.10,-0.5,6,20,Произвольная программа,q Прыжок приземлён в четверть x Надбавка за пр...,20,71.66
21297,462406,1995,7116,23.20,25.50,46.22,21.22,-0.5,7,21,Произвольная программа,q Прыжок приземлён в четверть e Явно неправиль...,21,71.27
21298,462407,34841,7116,21.83,25.00,45.46,20.96,-0.5,5,22,Произвольная программа,q Прыжок приземлён в четверть << Пониженный пр...,22,70.49
21299,462408,34980,7116,21.17,26.00,45.24,20.74,-1.5,2,23,Произвольная программа,q Прыжок приземлён в четверть < Недокрученный ...,24,68.22


In [17]:
# удаляем столбец 'starting_place',тк используется жеребьевка
total_scores_df = total_scores_df.drop(columns=['starting_place'])

In [18]:
total_scores_df.segment_name.value_counts()

segment_name
Произвольная программа     9485
Короткая программа         7877
Произвольная программа.    2428
Элементы                    601
Прыжки                      254
Пpoизвoльнaя пpoгpaммa      159
Кopoткaя пpoгpaммa          101
Кoроткая программа          100
Элeмeнты                     58
Kopoткaя пpoгpaммa           58
Произвольнaя программа       43
Прoизвольная прoграмма       38
Пpoизвольная программа       30
Коpоткая программа           18
Интерпретация.               17
Пpoизвольнaя программа        7
Интерпретация                 4
Name: count, dtype: int64

In [19]:
# нижний регистр
total_scores_df['segment_name'] = total_scores_df['segment_name'].str.lower() 
# удаление всех символов кроме букв и пробелов
total_scores_df['segment_name'] = total_scores_df['segment_name'].str.replace(r'[^\w\s]', '', regex=True) 
# удаление лишних пробелов 
total_scores_df['segment_name'] = total_scores_df['segment_name'].str.replace(r'\s+', ' ', regex=True)
# удаление пробелов в начале и конце строк  
total_scores_df['segment_name'] = total_scores_df['segment_name'].str.strip() 
# остается только первое слово
total_scores_df['segment_name'] = total_scores_df['segment_name'].str.split().str[0]  

total_scores_df['segment_name'].value_counts()

segment_name
произвольная     11913
короткая          7877
элементы           601
прыжки             254
пpoизвoльнaя       159
кopoткaя           101
кoроткая           100
элeмeнты            58
kopoткaя            58
произвольнaя        43
прoизвольная        38
пpoизвольная        30
интерпретация       21
коpоткая            18
пpoизвольнaя         7
Name: count, dtype: int64

In [20]:
replacements = {
    r'пpoизвoльнaя': 'произвольная',
    r'кopoткaя': 'короткая',
    r'кoроткая': 'короткая',
    r'элeмeнты': 'элементы',
    r'kopoткaя': 'короткая',
    r'произвольнaя': 'произвольная',
    r'прoизвольная': 'произвольная',
    r'пpoизвольная': 'произвольная',
    r'коpоткая': 'короткая',
    r'пpoизвольнaя': 'произвольная'
}

for pattern, replacement in replacements.items():
   total_scores_df['segment_name'] = total_scores_df['segment_name'].replace(pattern, replacement, regex=True)

total_scores_df['segment_name'].value_counts()

segment_name
произвольная     12190
короткая          8154
элементы           659
прыжки             254
интерпретация       21
Name: count, dtype: int64

**Рекомендация!**

Для улучшения качества данных и минимизации ошибок при вводе в поле segment_name рекомендую использовать выпадающий список с предопределенными категориями.

In [21]:
total_scores_df

,id,unit_id,tournament_id,base_score,components_score,total_score,elements_score,decreasings_score,place,segment_name,info,overall_place,overall_total_score
0,442027,304,4785,47.20,43.47,102.70,59.23,0.0,1,короткая,x Надбавка за прыжки во второй половине програ...,2,293.74
1,442028,604,4785,45.40,46.71,101.19,54.48,0.0,2,короткая,q Прыжок приземлён в четверть x Надбавка за пр...,1,294.75
2,442029,409,4785,44.00,46.82,99.20,52.38,0.0,3,короткая,q Прыжок приземлён в четверть x Надбавка за пр...,4,285.57
3,442030,524,4785,46.20,44.86,91.84,47.98,-1.0,4,короткая,q Прыжок приземлён в четверть x Надбавка за пр...,3,292.42
4,442031,412,4785,44.20,42.59,89.73,48.14,-1.0,5,короткая,q Прыжок приземлён в четверть x Надбавка за пр...,6,265.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21296,462405,34979,7116,21.67,26.02,46.62,21.10,-0.5,20,произвольная,q Прыжок приземлён в четверть x Надбавка за пр...,20,71.66
21297,462406,1995,7116,23.20,25.50,46.22,21.22,-0.5,21,произвольная,q Прыжок приземлён в четверть e Явно неправиль...,21,71.27
21298,462407,34841,7116,21.83,25.00,45.46,20.96,-0.5,22,произвольная,q Прыжок приземлён в четверть << Пониженный пр...,22,70.49
21299,462408,34980,7116,21.17,26.00,45.24,20.74,-1.5,23,произвольная,q Прыжок приземлён в четверть < Недокрученный ...,24,68.22


In [22]:
total_scores_df['overall_total_score'].describe()

count    21278.000000
mean       117.476497
std         66.310071
min          0.000000
25%         69.450000
50%        119.670000
75%        168.722500
max        323.050000
Name: overall_total_score, dtype: float64

In [23]:
# удалим все русские буквы и пробелы
def remove_russian_letters(text):
    if not isinstance(text, str):
        text = str(text)
    # удаляем русские буквы
    cleaned_text = re.sub(r'[а-яА-ЯёЁ]', '', text)
    # удаляем лишние пробелы
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

total_scores_df['error_symbol'] = total_scores_df['info'].apply(remove_russian_letters)

total_scores_df[['info', 'error_symbol']]

,info,error_symbol
0,x Надбавка за прыжки во второй половине програ...,x (10%)
1,q Прыжок приземлён в четверть x Надбавка за пр...,q x (10%)
2,q Прыжок приземлён в четверть x Надбавка за пр...,q x (10%)
3,q Прыжок приземлён в четверть x Надбавка за пр...,q x (10%) F
4,q Прыжок приземлён в четверть x Надбавка за пр...,q x (10%) F
...,...,...
21296,q Прыжок приземлён в четверть x Надбавка за пр...,q x (10%) F
21297,q Прыжок приземлён в четверть e Явно неправиль...,q e F/Lz x (10%) F
21298,q Прыжок приземлён в четверть << Пониженный пр...,q << ! F/Lz x (10%) F
21299,q Прыжок приземлён в четверть < Недокрученный ...,q < x (10%) F


In [24]:
# x Надбавка за прыжки во второй половине программы (10%)
# выделим бонус (символ x)
def extract_bonus(text):
    return 1 if 'x' in text else 0
    
total_scores_df['bonus'] = total_scores_df['error_symbol'].apply(extract_bonus)

# удаляем символ x из колонки error_symbol, чтобы оставить только ошибки
total_scores_df['error_symbol'] = total_scores_df['error_symbol'].str.replace('x', '').str.strip()

total_scores_df[['info', 'error_symbol', 'bonus']]

,info,error_symbol,bonus
0,x Надбавка за прыжки во второй половине програ...,(10%),1
1,q Прыжок приземлён в четверть x Надбавка за пр...,q (10%),1
2,q Прыжок приземлён в четверть x Надбавка за пр...,q (10%),1
3,q Прыжок приземлён в четверть x Надбавка за пр...,q (10%) F,1
4,q Прыжок приземлён в четверть x Надбавка за пр...,q (10%) F,1
...,...,...,...
21296,q Прыжок приземлён в четверть x Надбавка за пр...,q (10%) F,1
21297,q Прыжок приземлён в четверть e Явно неправиль...,q e F/Lz (10%) F,1
21298,q Прыжок приземлён в четверть << Пониженный пр...,q << ! F/Lz (10%) F,1
21299,q Прыжок приземлён в четверть < Недокрученный ...,q < (10%) F,1


In [25]:
# удаление подстроки (10%) 
total_scores_df['error_symbol'] = total_scores_df['error_symbol'].str.replace(r'\(10%\)', '', regex=True).str.strip()

In [26]:
# выделим символ q Прыжок приземлён в четверть
def extract_error(text):
    return 1 if 'q' in text else 0

total_scores_df['short_quarter_jump'] = total_scores_df['error_symbol'].apply(extract_error)

# удаляем символ q из колонки error_symbol, чтобы оставить только ошибки
total_scores_df['error_symbol'] = total_scores_df['error_symbol'].str.replace('q', '').str.strip()

total_scores_df[['info', 'error_symbol', 'short_quarter_jump']]

,info,error_symbol,short_quarter_jump
0,x Надбавка за прыжки во второй половине програ...,,0
1,q Прыжок приземлён в четверть x Надбавка за пр...,,1
2,q Прыжок приземлён в четверть x Надбавка за пр...,,1
3,q Прыжок приземлён в четверть x Надбавка за пр...,F,1
4,q Прыжок приземлён в четверть x Надбавка за пр...,F,1
...,...,...,...
21296,q Прыжок приземлён в четверть x Надбавка за пр...,F,1
21297,q Прыжок приземлён в четверть e Явно неправиль...,e F/Lz F,1
21298,q Прыжок приземлён в четверть << Пониженный пр...,<< ! F/Lz F,1
21299,q Прыжок приземлён в четверть < Недокрученный ...,< F,1


In [27]:
# удаление подстроки F/Lz
total_scores_df['error_symbol'] = total_scores_df['error_symbol'].str.replace(r'F/Lz', '', regex=True).str.strip()

In [28]:
# выделим символ F Падение в элементе
def extract_error(text):
    return 1 if 'F' in text else 0

# Применение функции к DataFrame для создания новой колонки, где 1 обозначает ошибку "падение"
total_scores_df['fall'] = total_scores_df['error_symbol'].apply(extract_error)

# Удаление символа 'F' из колонки error_symbol, чтобы оставить только другие ошибки
total_scores_df['error_symbol'] = total_scores_df['error_symbol'].str.replace('F', '').str.strip()


In [29]:
# извлечение '<' и '<<'
def extract_single_angle(text):
    return 1 if '<' in text and '<<' not in text else 0

def extract_double_angle(text):
    return 1 if '<<' in text else 0

# Применение функций к DataFrame
total_scores_df['uncoiled_jump_less90'] = total_scores_df['error_symbol'].apply(extract_single_angle)
total_scores_df['uncoiled_jump_more180'] = total_scores_df['error_symbol'].apply(extract_double_angle)

# Очистка колонки error_symbol
total_scores_df['error_symbol'] = total_scores_df['error_symbol'].replace({'<<': '', '<': ''}, regex=True).str.strip()

In [30]:
# выделим символ ! – Неясное ребро на толчке F/Lz
def extract_error(text):
    return 1 if '!' in text else 0

total_scores_df['obscure_edge'] = total_scores_df['error_symbol'].apply(extract_error)

# удаляем символ ! из колонки error_symbol, чтобы оставить только ошибки
total_scores_df['error_symbol'] = total_scores_df['error_symbol'].str.replace('!', '').str.strip()


In [31]:
# выделим символ @@@(1 of 6) – Нарушения в костюме
def extract_error(text):
    return 1 if '@@@(1 of 6)' in text else 0

total_scores_df['violations_suit'] = total_scores_df['error_symbol'].apply(extract_error)

# удаляем символ , чтобы оставить только ошибки
total_scores_df['error_symbol'] = total_scores_df['error_symbol'].str.replace('@@@(1 of 6)', '').str.strip()

In [32]:
# выделим символ @@@0.00 – Снижения@@@0.00
def extract_error(text):
    return 1 if '@@@0.00' in text else 0

total_scores_df['lower_score'] = total_scores_df['error_symbol'].apply(extract_error)

# удаляем символ чтобы оставить только ошибки
total_scores_df['error_symbol'] = total_scores_df['error_symbol'].str.replace('@@@0.00', '').str.strip()

In [33]:
# выделим символ * Недопустимый элемен
def extract_error(text):
    return 1 if '*' in text else 0

total_scores_df['invalid_element'] = total_scores_df['error_symbol'].apply(extract_error)

# удаляем символ, чтобы оставить только ошибки
total_scores_df['error_symbol'] = total_scores_df['error_symbol'].str.replace('*', '').str.strip()

In [34]:
# выделим символ nS Нет базовой позиции 'волчок'
def extract_error(text):
    return 1 if 'nS' in text else 0

total_scores_df['no_spinning_top'] = total_scores_df['error_symbol'].apply(extract_error)

# удаляем символ, чтобы оставить только ошибки
total_scores_df['error_symbol'] = total_scores_df['error_symbol'].str.replace('nS', '').str.strip()

In [35]:
total_scores_df['error_symbol'] = total_scores_df['error_symbol'].str.replace("'", "")

In [36]:
# выделим символ e Явно неправильное ребро на толчке F/Lz
def extract_error(text):
    return 1 if 'e' in text else 0

total_scores_df['wrong_edge_push'] = total_scores_df['error_symbol'].apply(extract_error)

# удаляем символ, чтобы оставить только ошибки
total_scores_df['error_symbol'] = total_scores_df['error_symbol'].str.replace('e', '').str.strip()

In [37]:
# выделим символ REP Повторение прыжка не в каскаде
def extract_error(text):
    return 1 if 'REP' in text else 0

total_scores_df['repeat_jump'] = total_scores_df['error_symbol'].apply(extract_error)

# удаляем символ, чтобы оставить только ошибки
total_scores_df['error_symbol'] = total_scores_df['error_symbol'].str.replace('REP', '').str.strip()

In [38]:
# выделим символ nC Нет базовой позиции 'либела'
def extract_error(text):
    return 1 if 'nC' in text else 0

total_scores_df['no_libela'] = total_scores_df['error_symbol'].apply(extract_error)

# удаляем символ, чтобы оставить только ошибки
total_scores_df['error_symbol'] = total_scores_df['error_symbol'].str.replace('nC', '').str.strip()

In [39]:
# выделим символ Элемент получил бонус@@@Бонусных баллов: +1,00
def extract_error(text):
    return 1 if 'B @@@ : +1,00' in text else 0

total_scores_df['plus_one_bonus'] = total_scores_df['error_symbol'].apply(extract_error)

# удаляем символ, чтобы оставить только ошибки
total_scores_df['error_symbol'] = total_scores_df['error_symbol'].str.replace('B @@@ : +1,00', '').str.strip()

In [40]:
# выделим символ Элемент получил бонус@@@Бонусных баллов: +2,00
def extract_error(text):
    return 1 if 'B @@@ : +2,00' in text else 0

total_scores_df['plus_two_bonus'] = total_scores_df['error_symbol'].apply(extract_error)

# удаляем символ, чтобы оставить только ошибки
total_scores_df['error_symbol'] = total_scores_df['error_symbol'].str.replace('B @@@ : +2,00', '').str.strip()

In [41]:
# выделим символ Элемент получил бонус@@@Бонусных баллов: +3,00
def extract_error(text):
    return 1 if 'B @@@ : +3,00' in text else 0

total_scores_df['plus_three_bonus'] = total_scores_df['error_symbol'].apply(extract_error)

# удаляем символ, чтобы оставить только ошибки
total_scores_df['error_symbol'] = total_scores_df['error_symbol'].str.replace('B @@@ : +3,00', '').str.strip()

In [42]:
# выделим символ nU Нет базовой позиции 'стоя'
def extract_error(text):
    return 1 if 'nU' in text else 0

total_scores_df['no_standing_position'] = total_scores_df['error_symbol'].apply(extract_error)

# удаляем символ, чтобы оставить только ошибки
total_scores_df['error_symbol'] = total_scores_df['error_symbol'].str.replace('nU', '').str.strip()

In [43]:
# выделим символ Мастерство катания
def mark_mastery(text):
    if '@@@1.50@@@3' in text:
        return 1
    else:
        return 0

total_scores_df['skill_symbol'] = total_scores_df['error_symbol'].apply(mark_mastery)

total_scores_df['error_symbol'] = total_scores_df['error_symbol'].str.replace('@@@1.50@@@3', '').str.strip()

In [44]:
# выделим символ Снижения@@@Нарушение времени:@@@-0.50@@@-0.50
def extract_error(text):
    return 1 if '@@@ :@@@-0.50@@@-0.50' in text else 0

total_scores_df['time_violation'] = total_scores_df['error_symbol'].apply(extract_error)

# удаляем символ, чтобы оставить только ошибки
total_scores_df['error_symbol'] = total_scores_df['error_symbol'].str.replace('@@@ :@@@-0.50@@@-0.50', '').str.strip()

In [45]:
# удаляем символ, чтобы оставить только ошибки
total_scores_df['error_symbol'] = total_scores_df['error_symbol'].str.replace(':', '').str.strip()

In [46]:
# удаляем символ, чтобы оставить только ошибки
total_scores_df['error_symbol'] = total_scores_df['error_symbol'].str.replace('n', '').str.strip()

In [47]:
total_scores_df['skill_symbol'].value_counts()

skill_symbol
0    20670
1      625
Name: count, dtype: int64

In [48]:
total_scores_df['error_symbol'] = total_scores_df['error_symbol'].str.replace('@@@', '').str.strip()

In [49]:
# удаляем все числа и запятые из столбца 'error_symbol'
total_scores_df['error_symbol'] = total_scores_df['error_symbol'].str.replace('[0-9,]', '', regex=True).str.strip()

In [50]:
total_scores_df['error_symbol'].value_counts()

error_symbol
                19637
.                 625
a                 575
..                363
( of )             31
B                  24
+                  10
B   +               9
( of )DDD           6
( of )DD            4
-.-.                3
( of )DDDDD         3
( of )DDDD          2
. -.-.              1
.-------            1
( of )DDDDDD        1
Name: count, dtype: int64

In [51]:
# Фильтрация строк, где в колонке 'info' встречается символ 
filtered_df = total_scores_df[total_scores_df['info'].str.contains('50@@@3,50@@@3,50@@@3.50', na=False)]
pd.set_option('display.max_colwidth', None)
filtered_df['info']

83                     Мастерство катания@@@1.50@@@3,50@@@3,50@@@3,50@@@3.50
86                     Мастерство катания@@@1.50@@@3,50@@@3,50@@@3,50@@@3.50
88                     Мастерство катания@@@1.50@@@3,50@@@3,50@@@3,50@@@3.50
95                     Мастерство катания@@@1.50@@@3,50@@@3,50@@@3,50@@@3.50
117                    Мастерство катания@@@1.50@@@3,50@@@3,50@@@3,50@@@3.50
2513                   Мастерство катания@@@1.50@@@3,50@@@3,50@@@3,50@@@3.50
2771                   Мастерство катания@@@1.50@@@3,50@@@3,50@@@3,50@@@3.50
4166                   Мастерство катания@@@1.50@@@3,50@@@3,50@@@3,50@@@3.50
5471                   Мастерство катания@@@1.50@@@3,50@@@3,50@@@3,50@@@3.50
5992                   Мастерство катания@@@1.50@@@3,50@@@3,50@@@3,50@@@3.50
6009                   Мастерство катания@@@1.50@@@3,50@@@3,50@@@3,50@@@3.50
6010                   Мастерство катания@@@1.50@@@3,50@@@3,50@@@3,50@@@3.50
6022                   Мастерство катания@@@1.50@@@3,50@@@3,50@@@3,50@@@3.50

In [52]:
# удаляем столбец 'info',тк все необходимые данные были извлечены
total_scores_df = total_scores_df.drop(columns=['info'])

In [53]:
total_scores_df

,id,unit_id,tournament_id,base_score,components_score,total_score,elements_score,decreasings_score,place,segment_name,...,no_spinning_top,wrong_edge_push,repeat_jump,no_libela,plus_one_bonus,plus_two_bonus,plus_three_bonus,no_standing_position,skill_symbol,time_violation
0,442027,304,4785,47.20,43.47,102.70,59.23,0.0,1,короткая,...,0,0,0,0,0,0,0,0,0,0
1,442028,604,4785,45.40,46.71,101.19,54.48,0.0,2,короткая,...,0,0,0,0,0,0,0,0,0,0
2,442029,409,4785,44.00,46.82,99.20,52.38,0.0,3,короткая,...,0,0,0,0,0,0,0,0,0,0
3,442030,524,4785,46.20,44.86,91.84,47.98,-1.0,4,короткая,...,0,0,0,0,0,0,0,0,0,0
4,442031,412,4785,44.20,42.59,89.73,48.14,-1.0,5,короткая,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21296,462405,34979,7116,21.67,26.02,46.62,21.10,-0.5,20,произвольная,...,0,0,0,0,0,0,0,0,0,0
21297,462406,1995,7116,23.20,25.50,46.22,21.22,-0.5,21,произвольная,...,0,1,0,0,0,0,0,0,0,0
21298,462407,34841,7116,21.83,25.00,45.46,20.96,-0.5,22,произвольная,...,0,0,0,0,0,0,0,0,0,0
21299,462408,34980,7116,21.17,26.00,45.24,20.74,-1.5,23,произвольная,...,0,0,0,0,0,0,0,0,0,0


In [54]:
total_scores_df.isnull().sum()

id                        0
unit_id                   0
tournament_id             0
base_score                0
components_score          0
total_score               0
elements_score            0
decreasings_score         0
place                     0
segment_name             17
overall_place             0
overall_total_score      17
error_symbol              0
bonus                     0
short_quarter_jump        0
fall                      0
uncoiled_jump_less90      0
uncoiled_jump_more180     0
obscure_edge              0
violations_suit           0
lower_score               0
invalid_element           0
no_spinning_top           0
wrong_edge_push           0
repeat_jump               0
no_libela                 0
plus_one_bonus            0
plus_two_bonus            0
plus_three_bonus          0
no_standing_position      0
skill_symbol              0
time_violation            0
dtype: int64

In [55]:
total_scores_df['segment_name'].unique()

array(['короткая', 'произвольная', 'элементы', 'интерпретация', nan,
       'прыжки'], dtype=object)

In [56]:
total_scores_df['segment_name'] = total_scores_df['segment_name'].fillna('любители')

In [57]:
missing = total_scores_df[total_scores_df['overall_total_score'].isna()]
missing = missing.sort_values(by='overall_total_score')
missing

,id,unit_id,tournament_id,base_score,components_score,total_score,elements_score,decreasings_score,place,segment_name,...,no_spinning_top,wrong_edge_push,repeat_jump,no_libela,plus_one_bonus,plus_two_bonus,plus_three_bonus,no_standing_position,skill_symbol,time_violation
680,442631,9522,4862,0.0,15.33,15.33,0.0,0.0,1,любители,...,0,0,0,0,0,0,0,0,0,0
681,442632,9523,4862,0.0,13.92,13.92,0.0,0.0,2,любители,...,0,0,0,0,0,0,0,0,0,0
682,442633,9529,4862,0.0,12.58,12.58,0.0,0.0,3,любители,...,0,0,0,0,0,0,0,0,0,0
684,442634,9530,4862,0.0,13.24,13.24,0.0,0.0,1,любители,...,0,0,0,0,0,0,0,0,0,0
685,442635,9524,4862,0.0,11.99,11.99,0.0,0.0,1,любители,...,0,0,0,0,0,0,0,0,0,0
686,442636,9510,4862,0.0,16.00,16.00,0.0,0.0,1,любители,...,0,0,0,0,0,0,0,0,0,0
689,442639,9522,4862,0.0,15.33,15.33,0.0,0.0,1,любители,...,0,0,0,0,0,0,0,0,0,0
690,442640,9523,4862,0.0,13.92,13.92,0.0,0.0,2,любители,...,0,0,0,0,0,0,0,0,0,0
691,442641,9529,4862,0.0,12.58,12.58,0.0,0.0,3,любители,...,0,0,0,0,0,0,0,0,0,0
692,442642,9530,4862,0.0,13.24,13.24,0.0,0.0,1,любители,...,0,0,0,0,0,0,0,0,0,0


Пропуски в overall_total_score итоговая оценка за весь турнир наблюдается у любителей,при этом в base_score 0,эти данные можно восстановить.Чтобы восстановить base_score,понадобиться соеденить с другой таблицей и потом можно восстановить overall_total_score.

In [58]:
total_scores_df.to_csv('total_scores_df_final.csv', index=False)

In [58]:
tournament_scores_df = pd.read_csv('data/tournament_scores.csv')

tournament_scores_df

,id,total_score_id,title,decrease,base_score,goe,avg_score
0,1,1,2A,NaN,3.3,0.66,3.96
1,2,1,3F+3Lo,NaN,10.2,-0.11,10.09
2,3,1,3Lz,NaN,0.0,-1.30,5.19
3,4,1,CCoSp4,NaN,3.5,0.56,4.06
4,5,1,FCSp4,NaN,3.2,0.64,3.84
...,...,...,...,...,...,...,...
172153,352008,463407,CCoSp3,NaN,3.0,0.60,3.60
172154,352009,463407,2Lz,NaN,2.1,0.00,2.10
172155,352010,463407,2Lo,NaN,1.7,0.23,1.93
172156,352011,463407,StSq2,NaN,2.6,0.52,3.12


In [59]:
tournament_scores_df.isnull().sum()

id                     0
total_score_id         0
title                  0
decrease          130973
base_score             0
goe                    0
avg_score              0
dtype: int64

In [60]:
# строки, где значения в столбце 'base_score' равны 0
zero_base_scores = tournament_scores_df[tournament_scores_df['base_score'] == 0]
zero_base_scores

,id,total_score_id,title,decrease,base_score,goe,avg_score
2,3,1,3Lz,NaN,0.0,-1.30,5.19
8,9,8,3F,NaN,0.0,0.42,6.25
11,12,8,3Lzq+2T,q,0.0,-0.47,7.45
12,13,8,3S,NaN,0.0,0.43,5.16
20,21,19,3Lz+3T,NaN,0.0,1.53,12.64
...,...,...,...,...,...,...,...
171955,322970,460276,2Lz+1A+SЕQ,NaN,0.0,0.14,3.66
171956,322971,460276,2F+2Lo,NaN,0.0,0.24,4.09
172028,351883,463389,2F<<*,<<*,0.0,0.00,0.00
172101,351960,463400,1F*,*,0.0,0.00,0.00


In [61]:
def extract_lo_jumps(title):
    # извлечение прыжков Lo
    pattern = r'(\d+Lo[^+]*)(?=\+|$)'
    # извлекаем все прыжки Lo
    jumps = re.findall(pattern, title)
    # преобразования списка в строку
    jumps_string = ', '.join(jumps)  
    # удаление извлечённых прыжков из исходной строки
    new_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', new_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['Lo_jump', 'tail_title']] = tournament_scores_df['title'].apply(
    lambda x: pd.Series(extract_lo_jumps(x))
)

In [62]:
tournament_scores_df['Lo_jump'].value_counts()

Lo_jump
                151082
2Lo               6255
3Lo               5469
2Lo<              2046
1Lo               1749
                 ...  
3Lo<, 2Loq           1
2Loq, 2Lo*           1
2Lo<*, 2Lo<*         1
2Lo<<, 2Lo           1
3Loq, 2Lo<           1
Name: count, Length: 91, dtype: int64

In [63]:
selected_row = tournament_scores_df.loc[tournament_scores_df['id'] == 1454]

selected_row

,id,total_score_id,title,decrease,base_score,goe,avg_score,Lo_jump,tail_title
1457,1454,1453,2F<+2Lo<,F,0.0,-0.72,2.36,2Lo<,2F<


In [64]:
tournament_scores_df

,id,total_score_id,title,decrease,base_score,goe,avg_score,Lo_jump,tail_title
0,1,1,2A,NaN,3.3,0.66,3.96,,2A
1,2,1,3F+3Lo,NaN,10.2,-0.11,10.09,3Lo,3F
2,3,1,3Lz,NaN,0.0,-1.30,5.19,,3Lz
3,4,1,CCoSp4,NaN,3.5,0.56,4.06,,CCoSp4
4,5,1,FCSp4,NaN,3.2,0.64,3.84,,FCSp4
...,...,...,...,...,...,...,...,...,...
172153,352008,463407,CCoSp3,NaN,3.0,0.60,3.60,,CCoSp3
172154,352009,463407,2Lz,NaN,2.1,0.00,2.10,,2Lz
172155,352010,463407,2Lo,NaN,1.7,0.23,1.93,2Lo,
172156,352011,463407,StSq2,NaN,2.6,0.52,3.12,,StSq2


In [65]:
def extract_1Lo(jump_data):
    # поиск 1Lo и последующих знаков до запятой
    pattern = r'1Lo[^,]*'
    match = re.search(pattern, jump_data)
    if match:
        return match.group()  
    return '' 

def remove_1Lo(jump_data):
    # удаление 1Lo и последующих знаков до запятой
    new_jump_data = re.sub(r'1Lo[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['1Lo'] = tournament_scores_df['Lo_jump'].apply(extract_1Lo)

tournament_scores_df['Lo_jump'] = tournament_scores_df['Lo_jump'].apply(remove_1Lo)

По данным и наличию ошибок видно,что данные извлечены правильно. Далее создадим колонки с каждым отдельным прыжком и проставим 1 если прыжок выполнен, 2 если есть ошибка и 0 есди прыжок вообще не указан. В коде выше мы уже все ошибки выделили,те нам не нужно заново их извлекать.

По количеству допущенных ошибок видно,что частая ошибка  это недокрутил, мало,но все же есть критичная ошибка ушел под звезду, элемент не засчитывается.

In [66]:
def process_jump(data):
    # является ли содержимое точным '1Lo'
    if data.strip() == '1Lo':
        return 1
    elif re.search(r'1Lo[^+ ]', data.strip()):
        return 2
    else:
        return 0
tournament_scores_df['1Lo'] = tournament_scores_df['1Lo'].apply(process_jump)

In [67]:
def extract_2Lo(jump_data):
    # поиск 2Lo и последующих знаков до запятой
    pattern = r'2Lo[^,]*'
    match = re.search(pattern, jump_data)
    if match:
        return match.group()  
    return '' 

def remove_2Lo(jump_data):
    # удаление 2Lo и последующих знаков до запятой
    new_jump_data = re.sub(r'2Lo[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['2Lo'] = tournament_scores_df['Lo_jump'].apply(extract_2Lo)

tournament_scores_df['Lo_jump'] = tournament_scores_df['Lo_jump'].apply(remove_2Lo)

In [68]:
def process_jump(data):
    # является ли содержимое точным '2Lo'
    if data.strip() == '2Lo':
        return 1
    elif re.search(r'2Lo[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['2Lo'] = tournament_scores_df['2Lo'].apply(process_jump)

In [69]:
def extract_3Lo(jump_data):
    # поиск 3Lo и последующих знаков до запятой
    pattern = r'3Lo[^,]*'
    match = re.search(pattern, jump_data)
    if match:
        return match.group()  
    return '' 

def remove_3Lo(jump_data):
    # удаление 3Lo и последующих знаков до запятой
    new_jump_data = re.sub(r'3Lo[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['3Lo'] = tournament_scores_df['Lo_jump'].apply(extract_3Lo)

tournament_scores_df['Lo_jump'] = tournament_scores_df['Lo_jump'].apply(remove_3Lo)

In [70]:
def process_jump(data):
    # является ли содержимое точным '3Lo'
    if data.strip() == '3Lo':
        return 1
    elif re.search(r'3Lo[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['3Lo'] = tournament_scores_df['3Lo'].apply(process_jump)

In [71]:
def extract_4Lo(jump_data):
    # поиск 4Lo и последующих знаков до запятой
    pattern = r'4Lo[^,]*'
    match = re.search(pattern, jump_data)
    if match:
        return match.group()  
    return '' 

def remove_4Lo(jump_data):
    # удаление 4Lo и последующих знаков до запятой
    new_jump_data = re.sub(r'4Lo[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['4Lo'] = tournament_scores_df['Lo_jump'].apply(extract_4Lo)

tournament_scores_df['Lo_jump'] = tournament_scores_df['Lo_jump'].apply(remove_4Lo)

In [72]:
def process_jump(data):
    # является ли содержимое точным '4Lo'
    if data.strip() == '4Lo':
        return 1
    elif re.search(r'4Lo[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['4Lo'] = tournament_scores_df['4Lo'].apply(process_jump)

In [73]:
# удаляем столбец 'Lo_jump',тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['Lo_jump'])

In [74]:
def extract_Lz_jumps(title):
    # извлечение прыжков Lz 
    pattern = r'(\d+Lz[^+]*)(?=\+|$)'
    # извлекаем все прыжки Lz
    jumps = re.findall(pattern, title)
    # преобразования списка в строку
    jumps_string = ', '.join(jumps)  
    # удаление извлечённых прыжков из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['Lz_jump', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_Lz_jumps(x))
)

Создадим колонки с каждым отдельным прыжком и проставим 1 если прыжок выполнен,2 если есть ошибка и 0 если прыжок вообще не указан. В коде выше мы уже все ошибки выделили,те нам не нужно заново их извлекать.

По количеству допущенных ошибок в Лутце видно,что частая ошибка  это недокрутил, мало,но все же есть отрыв с нечеткого ребра и критичная ошибка ушел под звезду, элемент не засчитывается.

In [75]:
def extract_1Lz(jump_data):
    # поиск 1Lz и последующих знаков до запятой
    pattern = r'1Lz[^,]*'
    match = re.search(pattern, jump_data)
    if match:
        return match.group()  
    return '' 

def remove_1Lz(jump_data):
    # удаление 1Lz и последующих знаков до запятой
    new_jump_data = re.sub(r'1Lz[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['1Lz'] = tournament_scores_df['Lz_jump'].apply(extract_1Lz)

tournament_scores_df['Lz_jump'] = tournament_scores_df['Lz_jump'].apply(remove_1Lz)

In [76]:
def process_jump(data):
    # является ли содержимое точным '1Lz'
    if data.strip() == '1Lz':
        return 1
    elif re.search(r'1Lz[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['1Lz'] = tournament_scores_df['1Lz'].apply(process_jump)

In [77]:
def extract_2Lz(jump_data):
    # поиск 2Lz и последующих знаков до запятой
    pattern = r'2Lz[^,]*'
    match = re.search(pattern, jump_data)
    if match:
        return match.group()  
    return '' 

def remove_2Lz(jump_data):
    # удаление 2Lz и последующих знаков до запятой
    new_jump_data = re.sub(r'2Lz[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['2Lz'] = tournament_scores_df['Lz_jump'].apply(extract_2Lz)

tournament_scores_df['Lz_jump'] = tournament_scores_df['Lz_jump'].apply(remove_2Lz)

In [78]:
def process_jump(data):
    # является ли содержимое точным '2Lz'
    if data.strip() == '2Lz':
        return 1
    elif re.search(r'2Lz[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['2Lz'] = tournament_scores_df['2Lz'].apply(process_jump)

In [79]:
def extract_3Lz(jump_data):
    # поиск 3Lz и последующих знаков до запятой
    pattern = r'3Lz[^,]*'
    match = re.search(pattern, jump_data)
    if match:
        return match.group()  
    return '' 

def remove_3Lz(jump_data):
    # удаление 3Lz и последующих знаков до запятой
    new_jump_data = re.sub(r'3Lz[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['3Lz'] = tournament_scores_df['Lz_jump'].apply(extract_3Lz)

tournament_scores_df['Lz_jump'] = tournament_scores_df['Lz_jump'].apply(remove_3Lz)

In [80]:
def process_jump(data):
    # является ли содержимое точным '3Lz'
    if data.strip() == '3Lz':
        return 1
    elif re.search(r'3Lz[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['3Lz'] = tournament_scores_df['3Lz'].apply(process_jump)

In [81]:
def extract_4Lz(jump_data):
    # поиск 4Lz и последующих знаков до запятой
    pattern = r'4Lz[^,]*'
    match = re.search(pattern, jump_data)
    if match:
        return match.group()  
    return '' 

def remove_4Lz(jump_data):
    # удаление 4Lz и последующих знаков до запятой
    new_jump_data = re.sub(r'4Lz[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['4Lz'] = tournament_scores_df['Lz_jump'].apply(extract_4Lz)

tournament_scores_df['Lz_jump'] = tournament_scores_df['Lz_jump'].apply(remove_4Lz)

In [82]:
def process_jump(data):
    # является ли содержимое точным '4Lz'
    if data.strip() == '4Lz':
        return 1
    elif re.search(r'4Lz[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['4Lz'] = tournament_scores_df['4Lz'].apply(process_jump)

In [83]:
# удаляем столбец 'Lz_jump',тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['Lz_jump'])

In [84]:
def extract_F_jumps(title):
    # извлечение прыжков F Флипп 
    pattern = r'(\d+F[^+]*)(?=\+|$)'
    # извлекаем все прыжки F
    jumps = re.findall(pattern, title)
    # преобразования списка в строку
    jumps_string = ', '.join(jumps)  
    # удаление извлечённых прыжков из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['F_jump', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_F_jumps(x))
)

In [85]:
def extract_1F(jump_data):
    # поиск 1F и последующих знаков до запятой
    pattern = r'1F[^,]*'
    match = re.search(pattern, jump_data)
    if match:
        return match.group()  
    return '' 

def remove_1F(jump_data):
    # удаление 1F и последующих знаков до запятой
    new_jump_data = re.sub(r'1F[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['1F'] = tournament_scores_df['F_jump'].apply(extract_1F)

tournament_scores_df['F_jump'] = tournament_scores_df['F_jump'].apply(remove_1F)

In [86]:
def process_jump(data):
    # является ли содержимое точным '1F'
    if data.strip() == '1F':
        return 1
    elif re.search(r'1F[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['1F'] = tournament_scores_df['1F'].apply(process_jump)

In [87]:
def extract_2F(jump_data):
    # поиск 2F и последующих знаков до запятой
    pattern = r'2F[^,]*'
    match = re.search(pattern, jump_data)
    if match:
        return match.group()  
    return '' 

def remove_2F(jump_data):
    # удаление 2F и последующих знаков до запятой
    new_jump_data = re.sub(r'2F[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['2F'] = tournament_scores_df['F_jump'].apply(extract_2F)

tournament_scores_df['F_jump'] = tournament_scores_df['F_jump'].apply(remove_2F)

In [88]:
def process_jump(data):
    # является ли содержимое точным '2F'
    if data.strip() == '2F':
        return 1
    elif re.search(r'2F[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['2F'] = tournament_scores_df['2F'].apply(process_jump)

In [89]:
def extract_3F(jump_data):
    # поиск 3F и последующих знаков до запятой
    pattern = r'3F[^,]*'
    match = re.search(pattern, jump_data)
    if match:
        return match.group()  
    return '' 

def remove_3F(jump_data):
    # удаление 3F и последующих знаков до запятой
    new_jump_data = re.sub(r'3F[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['3F'] = tournament_scores_df['F_jump'].apply(extract_3F)

tournament_scores_df['F_jump'] = tournament_scores_df['F_jump'].apply(remove_3F)

In [90]:
def process_jump(data):
    # является ли содержимое точным '3F'
    if data.strip() == '3F':
        return 1
    elif re.search(r'3F[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['3F'] = tournament_scores_df['3F'].apply(process_jump)

In [91]:
def extract_4F(jump_data):
    # поиск 4F и последующих знаков до запятой
    pattern = r'4F[^,]*'
    match = re.search(pattern, jump_data)
    if match:
        return match.group()  
    return '' 

def remove_4F(jump_data):
    # удаление 4F и последующих знаков до запятой
    new_jump_data = re.sub(r'4F[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['4F'] = tournament_scores_df['F_jump'].apply(extract_4F)

tournament_scores_df['F_jump'] = tournament_scores_df['F_jump'].apply(remove_4F)

In [92]:
def process_jump(data):
    # является ли содержимое точным '4F'
    if data.strip() == '4F':
        return 1
    elif re.search(r'4F[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['4F'] = tournament_scores_df['4F'].apply(process_jump)

In [93]:
# удаляем столбец 'F_jump',тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['F_jump'])

In [94]:
def extract_A_jumps(title):
    # извлечение прыжков A Аксель 
    pattern = r'(\d+A[^+]*)(?=\+|$)'
    # извлекаем все прыжки A
    jumps = re.findall(pattern, title)
    # преобразования списка в строку
    jumps_string = ', '.join(jumps)  
    # удаление извлечённых прыжков из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['A_jump', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_A_jumps(x))
)

In [95]:
def extract_1A(jump_data):
    # поиск 1A и последующих знаков до запятой
    pattern = r'1A[^,]*'
    match = re.search(pattern, jump_data)
    if match:
        return match.group()  
    return '' 

def remove_1A(jump_data):
    # удаление 1A и последующих знаков до запятой
    new_jump_data = re.sub(r'1A[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['1A'] = tournament_scores_df['A_jump'].apply(extract_1A)

tournament_scores_df['A_jump'] = tournament_scores_df['A_jump'].apply(remove_1A)

In [96]:
def process_jump(data):
    # является ли содержимое точным '1A'
    if data.strip() == '1A':
        return 1
    elif re.search(r'1A[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['1A'] = tournament_scores_df['1A'].apply(process_jump)

In [97]:
def extract_2A(jump_data):
    # поиск 2A и последующих знаков до запятой
    pattern = r'2A[^,]*'
    match = re.search(pattern, jump_data)
    if match:
        return match.group()  
    return '' 

def remove_2A(jump_data):
    # удаление 2A и последующих знаков до запятой
    new_jump_data = re.sub(r'2A[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['2A'] = tournament_scores_df['A_jump'].apply(extract_2A)

tournament_scores_df['A_jump'] = tournament_scores_df['A_jump'].apply(remove_2A)

In [98]:
def process_jump(data):
    # является ли содержимое точным '2A'
    if data.strip() == '2A':
        return 1
    elif re.search(r'2A[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['2A'] = tournament_scores_df['2A'].apply(process_jump)

In [99]:
def extract_3A(jump_data):
    # поиск 3A и последующих знаков до запятой
    pattern = r'3A[^,]*'
    match = re.search(pattern, jump_data)
    if match:
        return match.group()  
    return '' 

def remove_3A(jump_data):
    # удаление 3A и последующих знаков до запятой
    new_jump_data = re.sub(r'3A[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['3A'] = tournament_scores_df['A_jump'].apply(extract_3A)

tournament_scores_df['A_jump'] = tournament_scores_df['A_jump'].apply(remove_3A)

In [100]:
def process_jump(data):
    # является ли содержимое точным '3A'
    if data.strip() == '3A':
        return 1
    elif re.search(r'3A[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['3A'] = tournament_scores_df['3A'].apply(process_jump)

In [101]:
# удаляем столбец 'A_jump',тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['A_jump'])

In [102]:
def extract_T_jumps(title):
    # извлечение прыжков T Тулуп 
    pattern = r'(\d+T[^+]*)(?=\+|$)'
    # извлекаем все прыжки T
    jumps = re.findall(pattern, title)
    # преобразования списка в строку
    jumps_string = ', '.join(jumps)  
    # удаление извлечённых прыжков из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['T_jump', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_T_jumps(x))
)

In [103]:
def extract_1T(jump_data):
    # поиск 1T и последующих знаков до запятой
    pattern = r'1T[^,]*'
    match = re.search(pattern, jump_data)
    if match:
        return match.group()  
    return '' 

def remove_1T(jump_data):
    # удаление 1T и последующих знаков до запятой
    new_jump_data = re.sub(r'1T[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['1T'] = tournament_scores_df['T_jump'].apply(extract_1T)

tournament_scores_df['T_jump'] = tournament_scores_df['T_jump'].apply(remove_1T)

In [104]:
def process_jump(data):
    # является ли содержимое точным '1T'
    if data.strip() == '1T':
        return 1
    elif re.search(r'1T[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['1T'] = tournament_scores_df['1T'].apply(process_jump)

In [105]:
def extract_2T(jump_data):
    # поиск 2T и последующих знаков до запятой
    pattern = r'2T[^,]*'
    match = re.search(pattern, jump_data)
    if match:
        return match.group()  
    return '' 

def remove_2T(jump_data):
    # удаление 2T и последующих знаков до запятой
    new_jump_data = re.sub(r'2T[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['2T'] = tournament_scores_df['T_jump'].apply(extract_2T)

tournament_scores_df['T_jump'] = tournament_scores_df['T_jump'].apply(remove_2T)

In [106]:
def process_jump(data):
    # является ли содержимое точным '2T'
    if data.strip() == '2T':
        return 1
    elif re.search(r'2T[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['2T'] = tournament_scores_df['2T'].apply(process_jump)

In [107]:
def extract_3T(jump_data):
    # поиск 3T и последующих знаков до запятой
    pattern = r'3T[^,]*'
    match = re.search(pattern, jump_data)
    if match:
        return match.group()  
    return '' 

def remove_3T(jump_data):
    # удаление 3T и последующих знаков до запятой
    new_jump_data = re.sub(r'3T[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['3T'] = tournament_scores_df['T_jump'].apply(extract_3T)

tournament_scores_df['T_jump'] = tournament_scores_df['T_jump'].apply(remove_3T)

In [108]:
def process_jump(data):
    # является ли содержимое точным '3T'
    if data.strip() == '3T':
        return 1
    elif re.search(r'3T[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['3T'] = tournament_scores_df['3T'].apply(process_jump)

In [109]:
def extract_4T(jump_data):
    # поиск 4T и последующих знаков до запятой
    pattern = r'4T[^,]*'
    match = re.search(pattern, jump_data)
    if match:
        return match.group()  
    return '' 

def remove_4T(jump_data):
    # удаление 4T и последующих знаков до запятой
    new_jump_data = re.sub(r'4T[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['4T'] = tournament_scores_df['T_jump'].apply(extract_4T)

tournament_scores_df['T_jump'] = tournament_scores_df['T_jump'].apply(remove_4T)

In [110]:
def process_jump(data):
    # является ли содержимое точным '4T'
    if data.strip() == '4T':
        return 1
    elif re.search(r'4T[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['4T'] = tournament_scores_df['4T'].apply(process_jump)

In [111]:
# удаляем столбец 'T_jump',тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['T_jump'])

In [112]:
def extract_S_jumps(title):
    # извлечение прыжков S Сальхов
    pattern = r'(\d+S[^+]*)(?=\+|$)'
    # извлекаем все прыжки S
    jumps = re.findall(pattern, title)
    # преобразования списка в строку
    jumps_string = ', '.join(jumps)  
    # удаление извлечённых прыжков из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['S_jump', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_S_jumps(x))
)

In [113]:
def extract_1S(jump_data):
    # поиск 1S и последующих знаков до запятой
    pattern = r'1S[^,]*'
    match = re.search(pattern, jump_data)
    if match:
        return match.group()  
    return '' 

def remove_1S(jump_data):
    # удаление 1S и последующих знаков до запятой
    new_jump_data = re.sub(r'1S[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['1S'] = tournament_scores_df['S_jump'].apply(extract_1S)

tournament_scores_df['S_jump'] = tournament_scores_df['S_jump'].apply(remove_1S)

In [114]:
def process_jump(data):
    # является ли содержимое точным '1S'
    if data.strip() == '1S':
        return 1
    elif re.search(r'1S[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['1S'] = tournament_scores_df['1S'].apply(process_jump)

In [115]:
def extract_2S(jump_data):
    # поиск 2S и последующих знаков до запятой
    pattern = r'2S[^,]*'
    match = re.search(pattern, jump_data)
    if match:
        return match.group()  
    return '' 

def remove_2S(jump_data):
    # удаление 2S и последующих знаков до запятой
    new_jump_data = re.sub(r'2S[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['2S'] = tournament_scores_df['S_jump'].apply(extract_2S)

tournament_scores_df['S_jump'] = tournament_scores_df['S_jump'].apply(remove_2S)

In [116]:
def process_jump(data):
    # является ли содержимое точным '2S'
    if data.strip() == '2S':
        return 1
    elif re.search(r'2S[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['2S'] = tournament_scores_df['2S'].apply(process_jump)

In [117]:
def extract_3S(jump_data):
    # поиск 3S и последующих знаков до запятой
    pattern = r'3S[^,]*'
    match = re.search(pattern, jump_data)
    if match:
        return match.group()  
    return '' 

def remove_3S(jump_data):
    # удаление 3S и последующих знаков до запятой
    new_jump_data = re.sub(r'3S[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['3S'] = tournament_scores_df['S_jump'].apply(extract_3S)

tournament_scores_df['S_jump'] = tournament_scores_df['S_jump'].apply(remove_3S)

In [118]:
def process_jump(data):
    # является ли содержимое точным '3S'
    if data.strip() == '3S':
        return 1
    elif re.search(r'3S[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['3S'] = tournament_scores_df['3S'].apply(process_jump)

In [119]:
def extract_4S(jump_data):
    # поиск 4S и последующих знаков до запятой
    pattern = r'4S[^,]*'
    match = re.search(pattern, jump_data)
    if match:
        return match.group()  
    return '' 

def remove_4S(jump_data):
    # удаление 4S и последующих знаков до запятой
    new_jump_data = re.sub(r'4S[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['4S'] = tournament_scores_df['S_jump'].apply(extract_4S)

tournament_scores_df['S_jump'] = tournament_scores_df['S_jump'].apply(remove_4S)

In [120]:
def process_jump(data):
    # является ли содержимое точным '4S'
    if data.strip() == '4S':
        return 1
    elif re.search(r'4S[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['4S'] = tournament_scores_df['4S'].apply(process_jump)

In [121]:
# удаляем столбец 'S_jump',тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['S_jump'])

In [122]:
def extract_CCoSp(title):
    # извлечение CCoSp 
    pattern = r'(CCoSp[^+]*)(?=\+|$)'
    # извлекаем все CCoSp комбинации
    jumps = re.findall(pattern, title)
    # преобразуем список в строку
    jumps_string = ', '.join(jumps)
    # удаление извлеченных комбинаций из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['CCoSp_elements', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_CCoSp(x))
)

In [123]:
def extract_CCoSp1(jump_data):
    # поиск CCoSp1 и последующих знаков до запятой
    pattern = r'CCoSp1[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CCoSp1(jump_data):
    # удаление CCoSp1 и последующих знаков до запятой
    new_jump_data = re.sub(r'CCoSp1[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CCoSp1'] = tournament_scores_df['CCoSp_elements'].apply(extract_CCoSp1)
tournament_scores_df['CCoSp_elements'] = tournament_scores_df['CCoSp_elements'].apply(remove_CCoSp1)

In [124]:
def process_elements(data):
    # является ли содержимое точным CCoSp1
    if data.strip() == 'CCoSp1':
        return 1
    elif re.search(r'CCoSp1[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['CCoSp1'] = tournament_scores_df['CCoSp1'].apply(process_elements)

In [125]:
def extract_CCoSp2(jump_data):
    # поиск CCoSp2 и последующих знаков до запятой
    pattern = r'CCoSp2[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CCoSp2(jump_data):
    # удаление CCoSp2 и последующих знаков до запятой
    new_jump_data = re.sub(r'CCoSp2[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CCoSp2'] = tournament_scores_df['CCoSp_elements'].apply(extract_CCoSp2)
tournament_scores_df['CCoSp_elements'] = tournament_scores_df['CCoSp_elements'].apply(remove_CCoSp2)


In [126]:
def process_elements(data):
    # является ли содержимое точным CCoSp2
    if data.strip() == 'CCoSp2':
        return 1
    elif re.search(r'CCoSp2[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['CCoSp2'] = tournament_scores_df['CCoSp2'].apply(process_elements)

In [127]:
def extract_CCoSp3(jump_data):
    # поиск CCoSp3 и последующих знаков до запятой
    pattern = r'CCoSp3[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CCoSp3(jump_data):
    # удаление CCoSp3 и последующих знаков до запятой
    new_jump_data = re.sub(r'CCoSp3[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CCoSp3'] = tournament_scores_df['CCoSp_elements'].apply(extract_CCoSp3)
tournament_scores_df['CCoSp_elements'] = tournament_scores_df['CCoSp_elements'].apply(remove_CCoSp3)

In [128]:
def process_elements(data):
    # является ли содержимое точным CCoSp3
    if data.strip() == 'CCoSp3':
        return 1
    elif re.search(r'CCoSp3[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['CCoSp3'] = tournament_scores_df['CCoSp3'].apply(process_elements)

In [129]:
def extract_CCoSp4(jump_data):
    # поиск CCoSp4 и последующих знаков до запятой
    pattern = r'CCoSp4[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CCoSp4(jump_data):
    # удаление CCoSp4 и последующих знаков до запятой
    new_jump_data = re.sub(r'CCoSp4[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CCoSp4'] = tournament_scores_df['CCoSp_elements'].apply(extract_CCoSp4)
tournament_scores_df['CCoSp_elements'] = tournament_scores_df['CCoSp_elements'].apply(remove_CCoSp4)

In [130]:
def process_elements(data):
    # является ли содержимое точным CCoSp4
    if data.strip() == 'CCoSp4':
        return 1
    elif re.search(r'CCoSp4[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['CCoSp4'] = tournament_scores_df['CCoSp4'].apply(process_elements)

In [131]:
def extract_CCoSpB(jump_data):
    # поиск CCoSpB (B – базовый) и последующих знаков до запятой
    pattern = r'CCoSpB[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CCoSpB(jump_data):
    # удаление CCoSpB и последующих знаков до запятой
    new_jump_data = re.sub(r'CCoSpB[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CCoSpB'] = tournament_scores_df['CCoSp_elements'].apply(extract_CCoSpB)
tournament_scores_df['CCoSp_elements'] = tournament_scores_df['CCoSp_elements'].apply(remove_CCoSpB)

In [132]:
def process_elements(data):
    # является ли содержимое точным CCoSpB
    if data.strip() == 'CCoSpB':
        return 1
    elif re.search(r'CCoSpB[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['CCoSpB'] = tournament_scores_df['CCoSpB'].apply(process_elements)

In [133]:
def extract_CCoSp(jump_data):
    # поиск CCoSp и последующих знаков до запятой
    pattern = r'CCoSp[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CCoSp(jump_data):
    # удаление CCoSp и последующих знаков до запятой
    new_jump_data = re.sub(r'CCoSp[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CCoSp'] = tournament_scores_df['CCoSp_elements'].apply(extract_CCoSp)
tournament_scores_df['CCoSp_elements'] = tournament_scores_df['CCoSp_elements'].apply(remove_CCoSp)

In [134]:
def process_elements(data):
    # является ли содержимое точным CCoSp
    if data.strip() == 'CCoSp':
        return 1
    elif re.search(r'CCoSp[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['CCoSp'] = tournament_scores_df['CCoSp'].apply(process_elements)

In [135]:
tournament_scores_df['CCoSp'].value_counts()

CCoSp
0    171906
1       249
2         3
Name: count, dtype: int64

In [136]:
# удаляем столбец 'CCoSp_elements',тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['CCoSp_elements'])

In [137]:
def extract_CoSp(title):
    # извлечение CoSp Без смены ноги 
    pattern = r'(CoSp[^+]*)(?=\+|$)'
    # извлекаем все CoSp комбинации
    jumps = re.findall(pattern, title)
    # преобразуем список в строку
    jumps_string = ', '.join(jumps)
    # удаление извлеченных комбинаций из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['CoSp_elements', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_CoSp(x))
)

In [138]:
def extract_CoSpB(jump_data):
    # поиск CoSpB -базовый и последующих знаков до запятой
    pattern = r'CoSpB[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CoSpB(jump_data):
    # удаление CoSpB и последующих знаков до запятой
    new_jump_data = re.sub(r'CoSpB[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CoSpB'] = tournament_scores_df['CoSp_elements'].apply(extract_CoSpB)
tournament_scores_df['CoSp_elements'] = tournament_scores_df['CoSp_elements'].apply(remove_CoSpB)

In [139]:
def process_elements(data):
    # является ли содержимое точным CoSpB
    if data.strip() == 'CoSpB':
        return 1
    elif re.search(r'CoSpB[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['CoSpB'] = tournament_scores_df['CoSpB'].apply(process_elements)

In [140]:
def extract_CoSp1(jump_data):
    # поиск CoSp1 и последующих знаков до запятой
    pattern = r'CoSp1[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CoSp1(jump_data):
    # удаление CoSp1 и последующих знаков до запятой
    new_jump_data = re.sub(r'CoSp1[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CoSp1'] = tournament_scores_df['CoSp_elements'].apply(extract_CoSp1)
tournament_scores_df['CoSp_elements'] = tournament_scores_df['CoSp_elements'].apply(remove_CoSp1)

In [141]:
def process_elements(data):
    # является ли содержимое точным CoSp1
    if data.strip() == 'CoSp1':
        return 1
    elif re.search(r'CoSp1[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['CoSp1'] = tournament_scores_df['CoSp1'].apply(process_elements)

In [142]:
def extract_CoSp2(jump_data):
    # поиск CoSp2 и последующих знаков до запятой
    pattern = r'CoSp2[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CoSp2(jump_data):
    # удаление CoSp2 и последующих знаков до запятой
    new_jump_data = re.sub(r'CoSp2[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CoSp2'] = tournament_scores_df['CoSp_elements'].apply(extract_CoSp2)
tournament_scores_df['CoSp_elements'] = tournament_scores_df['CoSp_elements'].apply(remove_CoSp2)

In [143]:
def process_elements(data):
    # является ли содержимое точным CoSp2
    if data.strip() == 'CoSp2':
        return 1
    elif re.search(r'CoSp2[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['CoSp2'] = tournament_scores_df['CoSp2'].apply(process_elements)

In [144]:
def extract_CoSp(jump_data):
    # поиск CoSp и последующих знаков до запятой
    pattern = r'CoSp[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CoSp(jump_data):
    # удаление CoSp и последующих знаков до запятой
    new_jump_data = re.sub(r'CoSp[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CoSp'] = tournament_scores_df['CoSp_elements'].apply(extract_CoSp)
tournament_scores_df['CoSp_elements'] = tournament_scores_df['CoSp_elements'].apply(remove_CoSp)

In [145]:
def process_elements(data):
    # является ли содержимое точным CoSp
    if data.strip() == 'CoSp':
        return 1
    elif re.search(r'CoSp[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['CoSp'] = tournament_scores_df['CoSp'].apply(process_elements)

In [146]:
# удаляем столбец 'CoSp_elements',тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['CoSp_elements'])

In [147]:
def extract_ChSq(title):
    # извлечение ChSq с числом и возможной ошибкой
    pattern = r'(ChSq\d*[<>!ex*]?[^+]*)(?=\+|$)'
    # извлекаем все ChSq комбинации
    jumps = re.findall(pattern, title)
    # преобразуем список в строку
    jumps_string = ', '.join(jumps)
    # удаление извлеченных комбинаций из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['ChSq_elements', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_ChSq(x))
)

In [148]:
def extract_ChSq1(jump_data):
    # поиск ChSq1 и последующих знаков до запятой
    pattern = r'ChSq1[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_ChSq1(jump_data):
    # удаление ChSq1 и последующих знаков до запятой
    new_jump_data = re.sub(r'ChSq1[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['ChSq1'] = tournament_scores_df['ChSq_elements'].apply(extract_ChSq1)
tournament_scores_df['ChSq_elements'] = tournament_scores_df['ChSq_elements'].apply(remove_ChSq1)

In [149]:
def process_elements(data):
    # очищаем данные от пробелов и проверяем на точное совпадение с "ChSq1"
    if data.strip() == 'ChSq1':
        return 1
    elif re.search(r'ChSq1[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['ChSq1'] = tournament_scores_df['ChSq1'].apply(process_elements)

In [150]:
def extract_ChSq(jump_data):
    # поиск ChSq и последующих знаков до запятой
    pattern = r'ChSq[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_ChSq(jump_data):
    # удаление ChSq1 и последующих знаков до запятой
    new_jump_data = re.sub(r'ChSq[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['ChSq'] = tournament_scores_df['ChSq_elements'].apply(extract_ChSq)
tournament_scores_df['ChSq_elements'] = tournament_scores_df['ChSq_elements'].apply(remove_ChSq)

In [151]:
def process_elements(data):
    # очищаем данные от пробелов и проверяем на точное совпадение с "ChSq"
    if data.strip() == 'ChSq':
        return 1
    elif re.search(r'ChSq[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['ChSq'] = tournament_scores_df['ChSq'].apply(process_elements)

In [152]:
# удаляем столбец 'ChSq_elements',тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['ChSq_elements'])

In [153]:
def extract_SЕQ(title):
    # извлечение SЕQ – Комбинация прыжков с числом и возможной ошибкой
    pattern = r'(SЕQ\d*[<>!ex*]?[^+]*)(?=\+|$)'
    # извлекаем все SЕQ комбинации
    jumps = re.findall(pattern, title)
    # преобразуем список в строку
    jumps_string = ', '.join(jumps)
    # удаление извлеченных комбинаций из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['SЕQ_elements', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_SЕQ(x))
)

In [154]:
def extract_SЕQ(jump_data):
    # поиск SЕQ и последующих знаков до запятой
    pattern = r'SЕQ[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_SЕQ(jump_data):
    # удаление SЕQ и последующих знаков до запятой
    new_jump_data = re.sub(r'SЕQ[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['SЕQ'] = tournament_scores_df['SЕQ_elements'].apply(extract_SЕQ)
tournament_scores_df['SЕQ_elements'] = tournament_scores_df['SЕQ_elements'].apply(remove_SЕQ)

In [155]:
def process_elements(data):
    # очищаем данные от пробелов и проверяем на точное совпадение с SЕQ
    if data.strip() == 'SЕQ':
        return 1
    elif re.search(r'SЕQ[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['SЕQ'] = tournament_scores_df['SЕQ'].apply(process_elements)

In [156]:
# удаляем столбец SЕQ_elements,тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['SЕQ_elements'])

In [157]:
def extract_StSq(title):
    # извлечение StSq – Шаговая последовательность с числом и возможной ошибкой
    pattern = r'(StSq\d*[<>!ex*]?[^+]*)(?=\+|$)'
    # извлекаем все StSq комбинации
    jumps = re.findall(pattern, title)
    # преобразуем список в строку
    jumps_string = ', '.join(jumps)
    # удаление извлеченных комбинаций из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['StSq_elements', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_StSq(x))
)

In [158]:
def extract_StSqB(jump_data):
    # поиск StSqB и последующих знаков до запятой
    pattern = r'StSqB[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_StSqB(jump_data):
    # удаление StSqB и последующих знаков до запятой
    new_jump_data = re.sub(r'StSqB[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['StSqB'] = tournament_scores_df['StSq_elements'].apply(extract_StSqB)
tournament_scores_df['StSq_elements'] = tournament_scores_df['StSq_elements'].apply(remove_StSqB)

In [159]:
def process_elements(data):
    # очищаем данные от пробелов и проверяем на точное совпадение с StSqB
    if data.strip() == 'StSqB':
        return 1
    elif re.search(r'StSqB[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['StSqB'] = tournament_scores_df['StSqB'].apply(process_elements)

In [160]:
def extract_StSq1(jump_data):
    # поиск StSq1 и последующих знаков до запятой
    pattern = r'StSq1[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_StSq1(jump_data):
    # удаление StSq1 и последующих знаков до запятой
    new_jump_data = re.sub(r'StSq1[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['StSq1'] = tournament_scores_df['StSq_elements'].apply(extract_StSq1)
tournament_scores_df['StSq_elements'] = tournament_scores_df['StSq_elements'].apply(remove_StSq1)

In [161]:
def process_elements(data):
    # очищаем данные от пробелов и проверяем на точное совпадение с StSq1
    if data.strip() == 'StSq1':
        return 1
    elif re.search(r'StSq1[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['StSq1'] = tournament_scores_df['StSq1'].apply(process_elements)

In [162]:
def extract_StSq2(jump_data):
    # поиск StSq2 и последующих знаков до запятой
    pattern = r'StSq2[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_StSq2(jump_data):
    # удаление StSq2 и последующих знаков до запятой
    new_jump_data = re.sub(r'StSq2[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['StSq2'] = tournament_scores_df['StSq_elements'].apply(extract_StSq2)
tournament_scores_df['StSq_elements'] = tournament_scores_df['StSq_elements'].apply(remove_StSq2)

In [163]:
def process_elements(data):
    # очищаем данные от пробелов и проверяем на точное совпадение с StSq2
    if data.strip() == 'StSq2':
        return 1
    elif re.search(r'StSq2[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['StSq2'] = tournament_scores_df['StSq2'].apply(process_elements)

In [164]:
def extract_StSq3(jump_data):
    # поиск StSq3 и последующих знаков до запятой
    pattern = r'StSq3[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_StSq3(jump_data):
    # удаление StSq3 и последующих знаков до запятой
    new_jump_data = re.sub(r'StSq3[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['StSq3'] = tournament_scores_df['StSq_elements'].apply(extract_StSq3)
tournament_scores_df['StSq_elements'] = tournament_scores_df['StSq_elements'].apply(remove_StSq3)

In [165]:
def process_elements(data):
    # очищаем данные от пробелов и проверяем на точное совпадение с StSq3
    if data.strip() == 'StSq3':
        return 1
    elif re.search(r'StSq3[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['StSq3'] = tournament_scores_df['StSq3'].apply(process_elements)

In [166]:
def extract_StSq4(jump_data):
    # поиск StSq4 и последующих знаков до запятой
    pattern = r'StSq4[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_StSq4(jump_data):
    # удаление StSq4 и последующих знаков до запятой
    new_jump_data = re.sub(r'StSq4[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['StSq4'] = tournament_scores_df['StSq_elements'].apply(extract_StSq4)
tournament_scores_df['StSq_elements'] = tournament_scores_df['StSq_elements'].apply(remove_StSq4)

In [167]:
def process_elements(data):
    # очищаем данные от пробелов и проверяем на точное совпадение с StSq4
    if data.strip() == 'StSq4':
        return 1
    elif re.search(r'StSq4[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['StSq4'] = tournament_scores_df['StSq4'].apply(process_elements)

In [168]:
# удаляем столбец StSq_elements,тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['StSq_elements'])

In [169]:
def extract_LSp(title):
    # извлечение LSp – волчок с откинутой спиной с числом и возможной ошибкой
    pattern = r'(LSp\d*[<>!ex*]?[^+]*)(?=\+|$)'
    # извлекаем все LSp комбинации
    jumps = re.findall(pattern, title)
    # преобразуем список в строку
    jumps_string = ', '.join(jumps)
    # удаление извлеченных комбинаций из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['LSp_elements', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_LSp(x))
)

In [170]:
def extract_LSpB(jump_data):
    # поиск LSpB- базовый и последующих знаков до запятой
    pattern = r'LSpB[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_LSpB(jump_data):
    # удаление LSpB и последующих знаков до запятой
    new_jump_data = re.sub(r'LSpB[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['LSpB'] = tournament_scores_df['LSp_elements'].apply(extract_LSpB)
tournament_scores_df['LSp_elements'] = tournament_scores_df['LSp_elements'].apply(remove_LSpB)

In [171]:
def process_elements(data):
    # очищаем данные от пробелов и проверяем на точное совпадение с LSpB
    if data.strip() == 'LSpB':
        return 1
    elif re.search(r'LSpB[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['LSpB'] = tournament_scores_df['LSpB'].apply(process_elements)

In [172]:
def extract_LSp1(jump_data):
    # поиск LSp1 и последующих знаков до запятой
    pattern = r'LSp1[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_LSp1(jump_data):
    # удаление LSp1 и последующих знаков до запятой
    new_jump_data = re.sub(r'LSp1[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['LSp1'] = tournament_scores_df['LSp_elements'].apply(extract_LSp1)
tournament_scores_df['LSp_elements'] = tournament_scores_df['LSp_elements'].apply(remove_LSp1)

In [173]:
def process_elements(data):
    # очищаем данные от пробелов и проверяем на точное совпадение с LSp1
    if data.strip() == 'LSp1':
        return 1
    elif re.search(r'LSp1[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['LSp1'] = tournament_scores_df['LSp1'].apply(process_elements)

In [174]:
def extract_LSp2(jump_data):
    # поиск LSp2 и последующих знаков до запятой
    pattern = r'LSp2[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_LSp2(jump_data):
    # удаление LSp2 и последующих знаков до запятой
    new_jump_data = re.sub(r'LSp2[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['LSp2'] = tournament_scores_df['LSp_elements'].apply(extract_LSp2)
tournament_scores_df['LSp_elements'] = tournament_scores_df['LSp_elements'].apply(remove_LSp2)

In [175]:
def process_elements(data):
    # очищаем данные от пробелов и проверяем на точное совпадение с LSp2
    if data.strip() == 'LSp2':
        return 1
    elif re.search(r'LSp2[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['LSp2'] = tournament_scores_df['LSp2'].apply(process_elements)

In [176]:
def extract_LSp3(jump_data):
    # поиск LSp3 и последующих знаков до запятой
    pattern = r'LSp3[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_LSp3(jump_data):
    # удаление LSp3 и последующих знаков до запятой
    new_jump_data = re.sub(r'LSp3[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['LSp3'] = tournament_scores_df['LSp_elements'].apply(extract_LSp3)
tournament_scores_df['LSp_elements'] = tournament_scores_df['LSp_elements'].apply(remove_LSp3)

In [177]:
def process_elements(data):
    # очищаем данные от пробелов и проверяем на точное совпадение с LSp3
    if data.strip() == 'LSp3':
        return 1
    elif re.search(r'LSp3[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['LSp3'] = tournament_scores_df['LSp3'].apply(process_elements)

In [178]:
def extract_LSp4(jump_data):
    # поиск LSp4 и последующих знаков до запятой
    pattern = r'LSp4[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_LSp4(jump_data):
    # удаление LSp4 и последующих знаков до запятой
    new_jump_data = re.sub(r'LSp4[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['LSp4'] = tournament_scores_df['LSp_elements'].apply(extract_LSp4)
tournament_scores_df['LSp_elements'] = tournament_scores_df['LSp_elements'].apply(remove_LSp4)

In [179]:
def process_elements(data):
    # очищаем данные от пробелов и проверяем на точное совпадение с LSp4
    if data.strip() == 'LSp4':
        return 1
    elif re.search(r'LSp4[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['LSp4'] = tournament_scores_df['LSp4'].apply(process_elements)

In [180]:
# удаляем столбец LSp_elements,тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['LSp_elements'])

In [181]:
def extract_FCSp(title):
    # извлечение FCSp – прыжковое вращение в позиции верблюда с числом и возможной ошибкой
    pattern = r'(FCSp\d*[<>!ex*]?[^+]*)(?=\+|$)'
    # извлекаем все FCSp комбинации
    jumps = re.findall(pattern, title)
    # преобразуем список в строку
    jumps_string = ', '.join(jumps)
    # удаление извлеченных комбинаций из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['FCSp_elements', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_FCSp(x))
)

In [182]:
def extract_FCSpB(jump_data):
    # поиск FCSpB- базовый и последующих знаков до запятой
    pattern = r'FCSpB[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_FCSpB(jump_data):
    # удаление FCSpB и последующих знаков до запятой
    new_jump_data = re.sub(r'FCSpB[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['FCSpB'] = tournament_scores_df['FCSp_elements'].apply(extract_FCSpB)
tournament_scores_df['FCSp_elements'] = tournament_scores_df['FCSp_elements'].apply(remove_FCSpB)

In [183]:
def process_elements(data):
    # очищаем данные от пробелов и проверяем на точное совпадение с FCSpB
    if data.strip() == 'FCSpB':
        return 1
    elif re.search(r'FCSpB[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['FCSpB'] = tournament_scores_df['FCSpB'].apply(process_elements)

In [184]:
def extract_FCSp1(jump_data):
    # поиск FCSp1 и последующих знаков до запятой
    pattern = r'FCSp1[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_FCSp1(jump_data):
    # удаление FCSp1 и последующих знаков до запятой
    new_jump_data = re.sub(r'FCSp1[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['FCSp1'] = tournament_scores_df['FCSp_elements'].apply(extract_FCSp1)
tournament_scores_df['FCSp_elements'] = tournament_scores_df['FCSp_elements'].apply(remove_FCSp1)

In [185]:
def process_elements(data):
    # очищаем данные от пробелов и проверяем на точное совпадение с FCSp1
    if data.strip() == 'FCSp1':
        return 1
    elif re.search(r'FCSp1[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['FCSp1'] = tournament_scores_df['FCSp1'].apply(process_elements)

In [186]:
def extract_FCSp2(jump_data):
    # поиск FCSp2 и последующих знаков до запятой
    pattern = r'FCSp2[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_FCSp2(jump_data):
    # удаление FCSp2 и последующих знаков до запятой
    new_jump_data = re.sub(r'FCSp2[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['FCSp2'] = tournament_scores_df['FCSp_elements'].apply(extract_FCSp2)
tournament_scores_df['FCSp_elements'] = tournament_scores_df['FCSp_elements'].apply(remove_FCSp2)

In [187]:
def process_elements(data):
    # очищаем данные от пробелов и проверяем на точное совпадение с FCSp2
    if data.strip() == 'FCSp2':
        return 1
    elif re.search(r'FCSp2[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['FCSp2'] = tournament_scores_df['FCSp2'].apply(process_elements)

In [188]:
def extract_FCSp3(jump_data):
    # поиск FCSp3 и последующих знаков до запятой
    pattern = r'FCSp3[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_FCSp3(jump_data):
    # удаление FCSp3 и последующих знаков до запятой
    new_jump_data = re.sub(r'FCSp3[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['FCSp3'] = tournament_scores_df['FCSp_elements'].apply(extract_FCSp3)
tournament_scores_df['FCSp_elements'] = tournament_scores_df['FCSp_elements'].apply(remove_FCSp3)

In [189]:
def process_elements(data):
    # очищаем данные от пробелов и проверяем на точное совпадение с FCSp3
    if data.strip() == 'FCSp3':
        return 1
    elif re.search(r'FCSp3[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['FCSp3'] = tournament_scores_df['FCSp3'].apply(process_elements)

In [190]:
def extract_FCSp4(jump_data):
    # поиск FCSp4 и последующих знаков до запятой
    pattern = r'FCSp4[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_FCSp4(jump_data):
    # удаление FCSp4 и последующих знаков до запятой
    new_jump_data = re.sub(r'FCSp4[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['FCSp4'] = tournament_scores_df['FCSp_elements'].apply(extract_FCSp4)
tournament_scores_df['FCSp_elements'] = tournament_scores_df['FCSp_elements'].apply(remove_FCSp4)

In [191]:
def process_elements(data):
    # очищаем данные от пробелов и проверяем на точное совпадение с FCSp4
    if data.strip() == 'FCSp4':
        return 1
    elif re.search(r'FCSp4[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['FCSp4'] = tournament_scores_df['FCSp4'].apply(process_elements)

In [192]:
def extract_FCSp(jump_data):
    # поиск FCSp и последующих знаков до запятой
    pattern = r'FCSp[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_FCSp(jump_data):
    # удаление FCSp и последующих знаков до запятой
    new_jump_data = re.sub(r'FCSp[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['FCSp'] = tournament_scores_df['FCSp_elements'].apply(extract_FCSp)
tournament_scores_df['FCSp_elements'] = tournament_scores_df['FCSp_elements'].apply(remove_FCSp)

In [193]:
def process_elements(data):
    # очищаем данные от пробелов и проверяем на точное совпадение с FCSp
    if data.strip() == 'FCSp':
        return 1
    elif re.search(r'FCSp[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['FCSp'] = tournament_scores_df['FCSp'].apply(process_elements)

In [194]:
# удаляем столбец FCSp_elements,тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['FCSp_elements'])

In [195]:
def extract_FSSp(title):
    # извлечение FSSp – прыжковое вращение в позиции сидя с числом и возможной ошибкой
    pattern = r'(FSSp\d*[<>!ex*]?[^+]*)(?=\+|$)'
    # извлекаем все FSSp комбинации
    jumps = re.findall(pattern, title)
    # преобразуем список в строку
    jumps_string = ', '.join(jumps)
    # удаление извлеченных комбинаций из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['FSSp_elements', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_FSSp(x))
)

In [196]:
def extract_FSSpB(jump_data):
    # поиск FSSpB базовый и последующих знаков до запятой
    pattern = r'FSSpB[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_FSSpB(jump_data):
    # удаление FSSpB и последующих знаков до запятой
    new_jump_data = re.sub(r'FSSpB[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['FSSpB'] = tournament_scores_df['FSSp_elements'].apply(extract_FSSpB)
tournament_scores_df['FSSp_elements'] = tournament_scores_df['FSSp_elements'].apply(remove_FSSpB)

In [197]:
def process_elements(data):
    # очищаем данные от пробелов и проверяем на точное совпадение с FSSpB
    if data.strip() == 'FSSpB':
        return 1
    elif re.search(r'FSSpB[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['FSSpB'] = tournament_scores_df['FSSpB'].apply(process_elements)

In [198]:
def extract_FSSp1(jump_data):
    # поиск FSSp1 базовый и последующих знаков до запятой
    pattern = r'FSSp1[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_FSSp1(jump_data):
    # удаление FSSp1 и последующих знаков до запятой
    new_jump_data = re.sub(r'FSSp1[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['FSSp1'] = tournament_scores_df['FSSp_elements'].apply(extract_FSSp1)
tournament_scores_df['FSSp_elements'] = tournament_scores_df['FSSp_elements'].apply(remove_FSSp1)

In [199]:
def process_elements(data):
    # очищаем данные от пробелов и проверяем на точное совпадение с FSSp1
    if data.strip() == 'FSSp1':
        return 1
    elif re.search(r'FSSp1[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['FSSp1'] = tournament_scores_df['FSSp1'].apply(process_elements)

In [200]:
def extract_FSSp2(jump_data):
    # поиск FSSp2 базовый и последующих знаков до запятой
    pattern = r'FSSp2[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_FSSp2(jump_data):
    # удаление FSSp2 и последующих знаков до запятой
    new_jump_data = re.sub(r'FSSp2[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['FSSp2'] = tournament_scores_df['FSSp_elements'].apply(extract_FSSp2)
tournament_scores_df['FSSp_elements'] = tournament_scores_df['FSSp_elements'].apply(remove_FSSp2)

In [201]:
def process_elements(data):
    if data.strip() == 'FSSp2':
        return 1
    elif re.search(r'FSSp2[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['FSSp2'] = tournament_scores_df['FSSp2'].apply(process_elements)

In [202]:
def extract_FSSp3(jump_data):
    pattern = r'FSSp3[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_FSSp3(jump_data):
    new_jump_data = re.sub(r'FSSp3[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['FSSp3'] = tournament_scores_df['FSSp_elements'].apply(extract_FSSp3)
tournament_scores_df['FSSp_elements'] = tournament_scores_df['FSSp_elements'].apply(remove_FSSp3)

In [203]:
def process_elements(data):
    if data.strip() == 'FSSp3':
        return 1
    elif re.search(r'FSSp3[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['FSSp3'] = tournament_scores_df['FSSp3'].apply(process_elements)

In [204]:
def extract_FSSp4(jump_data):
    pattern = r'FSSp4[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_FSSp4(jump_data):
    new_jump_data = re.sub(r'FSSp4[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['FSSp4'] = tournament_scores_df['FSSp_elements'].apply(extract_FSSp4)
tournament_scores_df['FSSp_elements'] = tournament_scores_df['FSSp_elements'].apply(remove_FSSp4)

In [205]:
def process_elements(data):
    if data.strip() == 'FSSp4':
        return 1
    elif re.search(r'FSSp4[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['FSSp4'] = tournament_scores_df['FSSp4'].apply(process_elements)

In [206]:
def process_elements(data):
    if data.strip() == 'FSSp':
        return 1
    elif re.search(r'FSSp[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['FSSp'] = tournament_scores_df['FSSp_elements'].apply(process_elements)

In [207]:
# удаляем столбец FSSp_elements,тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['FSSp_elements'])

In [208]:
def extract_ChSpl(title):
    # извлечение ChSpl – хореографическая спираль с числом и возможной ошибкой
    pattern = r'(ChSpl\d*[<>!ex*]?[^+]*)(?=\+|$)'
    # извлекаем все ChSpl комбинации
    jumps = re.findall(pattern, title)
    # преобразуем список в строку
    jumps_string = ', '.join(jumps)
    # удаление извлеченных комбинаций из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['ChSpl_elements', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_ChSpl(x))
)

In [209]:
def extract_ChSpl1(jump_data):
    pattern = r'ChSpl1[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_ChSpl1(jump_data):
    new_jump_data = re.sub(r'ChSpl1[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['ChSpl1'] = tournament_scores_df['ChSpl_elements'].apply(extract_ChSpl1)
tournament_scores_df['ChSpl_elements'] = tournament_scores_df['ChSpl_elements'].apply(remove_ChSpl1)

In [210]:
def process_elements(data):
    if data.strip() == 'ChSpl1':
        return 1
    elif re.search(r'ChSpl1[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['ChSpl1'] = tournament_scores_df['ChSpl1'].apply(process_elements)

In [211]:
def process_elements(data):
    if data.strip() == 'ChSpl':
        return 1
    elif re.search(r'ChSpl[^+ ]', data.strip()):
        return 2
    else:
        return 0

tournament_scores_df['ChSpl'] = tournament_scores_df['ChSpl_elements'].apply(process_elements)

In [212]:
# удаляем столбец ChSpl_elements,тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['ChSpl_elements'])

In [213]:
def extract_CSSp (title):
    # извлечение CSSp – Сидя, волчок Sit Spin со сменой ноги
    pattern = r'(CSSp\d*[<>!ex*]?[^+]*)(?=\+|$)'
    # извлекаем все CSSp комбинации
    jumps = re.findall(pattern, title)
    # преобразуем список в строку
    jumps_string = ', '.join(jumps)
    # удаление извлеченных комбинаций из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['CSSp_elements', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_CSSp(x))
)

In [214]:
def extract_CSSpB (jump_data):
    pattern = r'CSSpB[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CSSpB(jump_data):
    new_jump_data = re.sub(r'CSSpB[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CSSpB'] = tournament_scores_df['CSSp_elements'].apply(extract_CSSpB)
tournament_scores_df['CSSp_elements'] = tournament_scores_df['CSSp_elements'].apply(remove_CSSpB)

In [215]:
def process_elements(data):
    data = data.strip()  
    if data == 'CSSpB':
        return 1
    elif re.search(r'^CSSpB[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['CSSpB'] = tournament_scores_df['CSSpB'].apply(process_elements)

In [216]:
def extract_CSSp1 (jump_data):
    pattern = r'CSSp1[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CSSp1(jump_data):
    new_jump_data = re.sub(r'CSSp1[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CSSp1'] = tournament_scores_df['CSSp_elements'].apply(extract_CSSp1)
tournament_scores_df['CSSp_elements'] = tournament_scores_df['CSSp_elements'].apply(remove_CSSp1)

In [217]:
def process_elements(data):
    data = data.strip()  
    if data == 'CSSp1':
        return 1
    elif re.search(r'^CSSp1[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['CSSp1'] = tournament_scores_df['CSSp1'].apply(process_elements)

In [218]:
def extract_CSSp2 (jump_data):
    pattern = r'CSSp2[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CSSp2(jump_data):
    new_jump_data = re.sub(r'CSSp2[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CSSp2'] = tournament_scores_df['CSSp_elements'].apply(extract_CSSp2)
tournament_scores_df['CSSp_elements'] = tournament_scores_df['CSSp_elements'].apply(remove_CSSp2)

In [219]:
def process_elements(data):
    data = data.strip()  
    if data == 'CSSp2':
        return 1
    elif re.search(r'^CSSp2[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['CSSp2'] = tournament_scores_df['CSSp2'].apply(process_elements)

In [220]:
def extract_CSSp3 (jump_data):
    pattern = r'CSSp3[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CSSp3(jump_data):
    new_jump_data = re.sub(r'CSSp3[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CSSp3'] = tournament_scores_df['CSSp_elements'].apply(extract_CSSp3)
tournament_scores_df['CSSp_elements'] = tournament_scores_df['CSSp_elements'].apply(remove_CSSp3)

In [221]:
def process_elements(data):
    data = data.strip()  
    if data == 'CSSp3':
        return 1
    elif re.search(r'^CSSp3[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['CSSp3'] = tournament_scores_df['CSSp3'].apply(process_elements)

In [222]:
def extract_CSSp4 (jump_data):
    pattern = r'CSSp4[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CSSp4(jump_data):
    new_jump_data = re.sub(r'CSSp4[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CSSp4'] = tournament_scores_df['CSSp_elements'].apply(extract_CSSp4)
tournament_scores_df['CSSp_elements'] = tournament_scores_df['CSSp_elements'].apply(remove_CSSp4)

In [223]:
def process_elements(data):
    data = data.strip()  
    if data == 'CSSp4':
        return 1
    elif re.search(r'^CSSp4[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['CSSp4'] = tournament_scores_df['CSSp4'].apply(process_elements)

In [224]:
def process_elements(data):
    data = data.strip()  
    if data == 'CSSp':
        return 1
    elif re.search(r'^CSSp[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['CSSp'] = tournament_scores_df['CSSp_elements'].apply(process_elements)

In [225]:
# удаляем столбец CSSp_elements,тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['CSSp_elements'])

In [226]:
def extract_F (title):
    # извлечение F – Флипп
    pattern = r'(F\d*[<>!ex*]?[^+]*)(?=\+|$)'
    # извлекаем все F комбинации
    jumps = re.findall(pattern, title)
    # преобразуем список в строку
    jumps_string = ', '.join(jumps)
    # удаление извлеченных комбинаций из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['F_elements', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_F(x))
)

In [227]:
def extract_F (jump_data):
    pattern = r'F[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_F(jump_data):
    new_jump_data = re.sub(r'F[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['F'] = tournament_scores_df['F_elements'].apply(extract_F)
tournament_scores_df['F_elements'] = tournament_scores_df['F_elements'].apply(remove_F)

In [228]:
def process_elements(data):
    data = data.strip()  
    if data == 'F':
        return 1
    else:
        return 0

tournament_scores_df['F'] = tournament_scores_df['F'].apply(process_elements)

In [229]:
# удаляем столбец F_elements,тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['F_elements'])

In [230]:
def extract_Eu(title):
    # извлечение Eu Европейский шаг 
    pattern = r'(\d+Eu[^+]*)(?=\+|$)'
    # извлекаем все Eu
    jumps = re.findall(pattern, title)
    # преобразования списка в строку
    jumps_string = ', '.join(jumps)  
    # удаление извлечённых Eu из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['Eu_jump', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_Eu(x))
)

In [231]:
def process_elements(data):
    data = data.strip()  
    if data == '1Eu':
        return 1
    elif re.search(r'^1Eu[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['1Eu'] = tournament_scores_df['Eu_jump'].apply(process_elements)

In [232]:
# удаляем столбец Eu_jump,тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['Eu_jump'])

In [233]:
def extract_COMBO (title):
    # извлечение COMBO обязательный каскад в короткой программе не был исполнен
    pattern = r'(COMBO\d*[<>!ex*]?[^+]*)(?=\+|$)'
    # извлекаем все COMBO комбинации
    jumps = re.findall(pattern, title)
    # преобразуем список в строку
    jumps_string = ', '.join(jumps)
    # удаление извлеченных комбинаций из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['COMBO_elements', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_COMBO(x))
)

In [234]:
def process_elements(data):
    data = data.strip()  
    if data == 'COMBO':
        return 1
    elif re.search(r'^COMBO[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['COMBO'] = tournament_scores_df['COMBO_elements'].apply(process_elements)

In [235]:
# удаляем столбец COMBO_elements,тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['COMBO_elements'])

In [236]:
def extract_REP (title):
    # извлечение REP Повтор,если по каким то причинам спортсмен оба раза прыгнул прыжок сольно, то ко второй попытке добавляют этот знак и базовую стоимость уменьшают на 30%
    pattern = r'(REP\d*[<>!ex*]?[^+]*)(?=\+|$)'
    # извлекаем все REP комбинации
    jumps = re.findall(pattern, title)
    # преобразуем список в строку
    jumps_string = ', '.join(jumps)
    # удаление извлеченных комбинаций из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['REP_elements', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_REP(x))
)

In [237]:
def process_elements(data):
    data = data.strip()  
    if data == 'REP':
        return 1
    elif re.search(r'^REP[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['REP'] = tournament_scores_df['REP_elements'].apply(process_elements)

In [238]:
# удаляем столбец REP_elements,тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['REP_elements'])

In [239]:
def extract_SSp (title):
    # извлечение SSp – Сидя, волчок Sit Spin в позиции
    pattern = r'(SSp\d*[<>!ex*]?[^+]*)(?=\+|$)'
    # извлекаем все SSp комбинации
    jumps = re.findall(pattern, title)
    # преобразуем список в строку
    jumps_string = ', '.join(jumps)
    # удаление извлеченных комбинаций из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['SSp_elements', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_SSp(x))
)

In [240]:
def extract_SSpB (jump_data):
    pattern = r'SSpB[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_SSpB(jump_data):
    new_jump_data = re.sub(r'SSpB[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['SSpB'] = tournament_scores_df['SSp_elements'].apply(extract_SSpB)
tournament_scores_df['SSp_elements'] = tournament_scores_df['SSp_elements'].apply(remove_SSpB)

In [241]:
def process_elements(data):
    data = data.strip()  
    if data == 'SSpB':
        return 1
    elif re.search(r'^SSpB[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['SSpB'] = tournament_scores_df['SSpB'].apply(process_elements)

In [242]:
def extract_SSp1 (jump_data):
    pattern = r'SSp1[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_SSp1(jump_data):
    new_jump_data = re.sub(r'SSp1[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['SSp1'] = tournament_scores_df['SSp_elements'].apply(extract_SSp1)
tournament_scores_df['SSp_elements'] = tournament_scores_df['SSp_elements'].apply(remove_SSp1)

In [243]:
def process_elements(data):
    data = data.strip()  
    if data == 'SSp1':
        return 1
    elif re.search(r'^SSp1[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['SSp1'] = tournament_scores_df['SSp1'].apply(process_elements)

In [244]:
def extract_SSp2 (jump_data):
    pattern = r'SSp2[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_SSp2(jump_data):
    new_jump_data = re.sub(r'SSp2[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['SSp2'] = tournament_scores_df['SSp_elements'].apply(extract_SSp2)
tournament_scores_df['SSp_elements'] = tournament_scores_df['SSp_elements'].apply(remove_SSp2)

In [245]:
def process_elements(data):
    data = data.strip()  
    if data == 'SSp2':
        return 1
    elif re.search(r'^SSp2[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['SSp2'] = tournament_scores_df['SSp2'].apply(process_elements)

In [246]:
def extract_SSp3 (jump_data):
    pattern = r'SSp3[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_SSp3(jump_data):
    new_jump_data = re.sub(r'SSp3[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['SSp3'] = tournament_scores_df['SSp_elements'].apply(extract_SSp3)
tournament_scores_df['SSp_elements'] = tournament_scores_df['SSp_elements'].apply(remove_SSp3)

In [247]:
def process_elements(data):
    data = data.strip()  
    if data == 'SSp3':
        return 1
    elif re.search(r'^SSp3[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['SSp3'] = tournament_scores_df['SSp3'].apply(process_elements)

In [248]:
def extract_SSp4 (jump_data):
    pattern = r'SSp4[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_SSp4(jump_data):
    new_jump_data = re.sub(r'SSp4[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['SSp4'] = tournament_scores_df['SSp_elements'].apply(extract_SSp4)
tournament_scores_df['SSp_elements'] = tournament_scores_df['SSp_elements'].apply(remove_SSp4)

In [249]:
def process_elements(data):
    data = data.strip()  
    if data == 'SSp4':
        return 1
    elif re.search(r'^SSp4[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['SSp4'] = tournament_scores_df['SSp4'].apply(process_elements)

In [250]:
def process_elements(data):
    data = data.strip()  
    if data == 'SSp':
        return 1
    elif re.search(r'^SSp[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['SSp'] = tournament_scores_df['SSp_elements'].apply(process_elements)

In [251]:
# удаляем столбец SSp_elements,тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['SSp_elements'])

In [252]:
def extract_CCSp (title):
    # извлечение CCSp – Либела Camel Spin со сменой ноги
    pattern = r'(CCSp\d*[<>!ex*]?[^+]*)(?=\+|$)'
    # извлекаем все CCSp комбинации
    jumps = re.findall(pattern, title)
    # преобразуем список в строку
    jumps_string = ', '.join(jumps)
    # удаление извлеченных комбинаций из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['CCSp_elements', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_CCSp(x))
)

In [253]:
def extract_CCSpB (jump_data):
    pattern = r'CCSpB[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CCSpB(jump_data):
    new_jump_data = re.sub(r'CCSpB[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CCSpB'] = tournament_scores_df['CCSp_elements'].apply(extract_CCSpB)
tournament_scores_df['CCSp_elements'] = tournament_scores_df['CCSp_elements'].apply(remove_CCSpB)

In [254]:
def process_elements(data):
    data = data.strip()  
    if data == 'CCSpB':
        return 1
    elif re.search(r'^CCSpB[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['CCSpB'] = tournament_scores_df['CCSpB'].apply(process_elements)

In [255]:
def extract_CCSp1 (jump_data):
    pattern = r'CCSp1[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CCSp1(jump_data):
    new_jump_data = re.sub(r'CCSp1[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CCSp1'] = tournament_scores_df['CCSp_elements'].apply(extract_CCSp1)
tournament_scores_df['CCSp_elements'] = tournament_scores_df['CCSp_elements'].apply(remove_CCSp1)

In [256]:
def process_elements(data):
    data = data.strip()  
    if data == 'CCSp1':
        return 1
    elif re.search(r'^CCSp1[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['CCSp1'] = tournament_scores_df['CCSp1'].apply(process_elements)

In [257]:
def extract_CCSp2 (jump_data):
    pattern = r'CCSp2[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CCSp2(jump_data):
    new_jump_data = re.sub(r'CCSp2[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CCSp2'] = tournament_scores_df['CCSp_elements'].apply(extract_CCSp2)
tournament_scores_df['CCSp_elements'] = tournament_scores_df['CCSp_elements'].apply(remove_CCSp2)

In [258]:
def process_elements(data):
    data = data.strip()  
    if data == 'CCSp2':
        return 1
    elif re.search(r'^CCSp2[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['CCSp2'] = tournament_scores_df['CCSp2'].apply(process_elements)

In [259]:
def extract_CCSp3 (jump_data):
    pattern = r'CCSp3[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CCSp3(jump_data):
    new_jump_data = re.sub(r'CCSp3[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CCSp3'] = tournament_scores_df['CCSp_elements'].apply(extract_CCSp3)
tournament_scores_df['CCSp_elements'] = tournament_scores_df['CCSp_elements'].apply(remove_CCSp3)

In [260]:
def process_elements(data):
    data = data.strip()  
    if data == 'CCSp3':
        return 1
    elif re.search(r'^CCSp3[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['CCSp3'] = tournament_scores_df['CCSp3'].apply(process_elements)

In [261]:
def extract_CCSp4 (jump_data):
    pattern = r'CCSp4[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CCSp4(jump_data):
    new_jump_data = re.sub(r'CCSp4[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CCSp4'] = tournament_scores_df['CCSp_elements'].apply(extract_CCSp4)
tournament_scores_df['CCSp_elements'] = tournament_scores_df['CCSp_elements'].apply(remove_CCSp4)

In [262]:
def process_elements(data):
    data = data.strip()  
    if data == 'CCSp4':
        return 1
    elif re.search(r'^CCSp4[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['CCSp4'] = tournament_scores_df['CCSp4'].apply(process_elements)

In [263]:
def process_elements(data):
    data = data.strip()  
    if data == 'CCSp':
        return 1
    elif re.search(r'^CCSp[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['CCSp'] = tournament_scores_df['CCSp_elements'].apply(process_elements)

In [264]:
# удаляем столбец CCSp_elements,тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['CCSp_elements'])

In [265]:
def extract_CSp (title):
    # извлечение CSp – Либела Camel Spin в позиции
    pattern = r'(CSp\d*[<>!ex*]?[^+]*)(?=\+|$)'
    # извлекаем все CSp комбинации
    jumps = re.findall(pattern, title)
    # преобразуем список в строку
    jumps_string = ', '.join(jumps)
    # удаление извлеченных комбинаций из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['CSp_elements', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_CSp(x))
)

In [266]:
def extract_CSpB (jump_data):
    pattern = r'CSpB[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CSpB (jump_data):
    new_jump_data = re.sub(r'CSpB[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CSpB'] = tournament_scores_df['CSp_elements'].apply(extract_CSpB)
tournament_scores_df['CSp_elements'] = tournament_scores_df['CSp_elements'].apply(remove_CSpB)

In [267]:
def process_elements(data):
    data = data.strip()  
    if data == 'CSpB':
        return 1
    elif re.search(r'^CSpB[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['CSpB'] = tournament_scores_df['CSpB'].apply(process_elements)

In [268]:
def extract_CSp1 (jump_data):
    pattern = r'CSp1[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CSp1 (jump_data):
    new_jump_data = re.sub(r'CSp1[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CSp1'] = tournament_scores_df['CSp_elements'].apply(extract_CSp1)
tournament_scores_df['CSp_elements'] = tournament_scores_df['CSp_elements'].apply(remove_CSp1)

In [269]:
def process_elements(data):
    data = data.strip()  
    if data == 'CSp1':
        return 1
    elif re.search(r'^CSp1[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['CSp1'] = tournament_scores_df['CSp1'].apply(process_elements)

In [270]:
def extract_CSp2 (jump_data):
    pattern = r'CSp2[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CSp2 (jump_data):
    new_jump_data = re.sub(r'CSp2[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CSp2'] = tournament_scores_df['CSp_elements'].apply(extract_CSp2)
tournament_scores_df['CSp_elements'] = tournament_scores_df['CSp_elements'].apply(remove_CSp2)

In [271]:
def process_elements(data):
    data = data.strip()  
    if data == 'CSp2':
        return 1
    elif re.search(r'^CSp2[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['CSp2'] = tournament_scores_df['CSp2'].apply(process_elements)

In [272]:
def extract_CSp3 (jump_data):
    pattern = r'CSp3[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CSp3 (jump_data):
    new_jump_data = re.sub(r'CSp3[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CSp3'] = tournament_scores_df['CSp_elements'].apply(extract_CSp3)
tournament_scores_df['CSp_elements'] = tournament_scores_df['CSp_elements'].apply(remove_CSp3)

In [273]:
def process_elements(data):
    data = data.strip()  
    if data == 'CSp3':
        return 1
    elif re.search(r'^CSp3[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['CSp3'] = tournament_scores_df['CSp3'].apply(process_elements)

In [274]:
def extract_CSp4 (jump_data):
    pattern = r'CSp4[^,]*'
    match = re.search(pattern, jump_data)
    return match.group() if match else ''

def remove_CSp4 (jump_data):
    new_jump_data = re.sub(r'CSp4[^,]*,?\s*', '', jump_data)
    return new_jump_data.strip(',')

tournament_scores_df['CSp4'] = tournament_scores_df['CSp_elements'].apply(extract_CSp4)
tournament_scores_df['CSp_elements'] = tournament_scores_df['CSp_elements'].apply(remove_CSp4)

In [275]:
def process_elements(data):
    data = data.strip()  
    if data == 'CSp4':
        return 1
    elif re.search(r'^CSp4[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['CSp4'] = tournament_scores_df['CSp4'].apply(process_elements)

In [276]:
def process_elements(data):
    data = data.strip()  
    if data == 'CSp':
        return 1
    elif re.search(r'^CSp[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['CSp'] = tournament_scores_df['CSp_elements'].apply(process_elements)

In [277]:
# удаляем столбец CSp_elements,тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['CSp_elements'])

In [278]:
def extract_SEQ (title):
    # извлечение SEQ
    pattern = r'(SEQ\d*[<>!ex*]?[^+]*)(?=\+|$)'
    # извлекаем все SEQ комбинации
    jumps = re.findall(pattern, title)
    # преобразуем список в строку
    jumps_string = ', '.join(jumps)
    # удаление извлеченных комбинаций из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['SEQ_elements', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_SEQ(x))
)

In [279]:
def process_elements(data):
    data = data.strip()  
    if data == 'SEQ':
        return 1
    elif re.search(r'^SEQ[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['SEQ_elements'] = tournament_scores_df['SEQ_elements'].apply(process_elements)

In [280]:
tournament_scores_df['SЕQ'] = tournament_scores_df['SЕQ'].astype(str)
tournament_scores_df['SEQ_elements'] = tournament_scores_df['SEQ_elements'].astype(str)
tournament_scores_df['SEQ'] = tournament_scores_df['SЕQ'] + ", " + tournament_scores_df['SEQ_elements']

In [281]:
def transform_seq(value):
    parts = value.split(', ')
    if parts == ['1', '0'] or parts == ['0', '1']:
        return 1
    elif parts == ['2', '0']:
        return 2
    else:
        return 0

tournament_scores_df['SEQ']= tournament_scores_df['SEQ'].apply(transform_seq)


In [282]:
tournament_scores_df = tournament_scores_df.drop(['SEQ_elements', 'SЕQ'], axis=1)


In [283]:
def extract_USp (title):
    # извлечение USp – Стоя Upright Spin в позиции
    pattern = r'(USp\d*[<>!ex*]?[^+]*)(?=\+|$)'
    # извлекаем все USp комбинации
    jumps = re.findall(pattern, title)
    # преобразуем список в строку
    jumps_string = ', '.join(jumps)
    # удаление извлеченных комбинаций из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['USp_elements', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_USp(x))
)

In [284]:
def process_elements(data):
    data = data.strip()  
    if data == 'USpB':
        return 1
    elif re.search(r'^USpB[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['USpB'] = tournament_scores_df['USp_elements'].apply(process_elements)

In [285]:
# удаляем столбец USp_elements,тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['USp_elements'])

In [286]:
def extract_Sp (title):
    # извлечение Sp – spin, его называют “винт”
    pattern = r'(Sp\d*[<>!ex*]?[^+]*)(?=\+|$)'
    # извлекаем все Sp комбинации
    jumps = re.findall(pattern, title)
    # преобразуем список в строку
    jumps_string = ', '.join(jumps)
    # удаление извлеченных комбинаций из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['Sp_elements', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_Sp(x))
)

In [287]:
def process_elements(data):
    data = data.strip()  
    if data == 'Sp':
        return 1
    elif re.search(r'^Sp[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['Sp'] = tournament_scores_df['Sp_elements'].apply(process_elements)

In [288]:
# удаляем столбец Sp_elements,тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['Sp_elements'])

In [289]:
def extract_A (title):
    # извлечение A - Аксель (англ. Axel jump) 
    pattern = r'(A\d*[<>!ex*]?[^+]*)(?=\+|$)'
    # извлекаем все A 
    jumps = re.findall(pattern, title)
    # преобразуем список в строку
    jumps_string = ', '.join(jumps)
    # удаление извлеченных комбинаций из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['A_elements', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_A(x))
)

In [290]:
def process_elements(data):
    data = data.strip()  
    if data == 'A':
        return 1
    elif re.search(r'^A[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['A'] = tournament_scores_df['A_elements'].apply(process_elements)

In [291]:
# удаляем столбец A_elements,тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['A_elements'])

In [292]:
def extract_Lo (title):
    # извлечение Lo - Риттбергер (Loop) обозначается как (Lo)- один из трёх рёберных прыжков в фигурном катании 
    pattern = r'(Lo\d*[<>!ex*]?[^+]*)(?=\+|$)'
    # извлекаем все Lo 
    jumps = re.findall(pattern, title)
    # преобразуем список в строку
    jumps_string = ', '.join(jumps)
    # удаление извлеченных комбинаций из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['Lo_elements', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_Lo(x))
)

In [293]:
def process_elements(data):
    data = data.strip()  
    if data == 'Lo':
        return 1
    elif re.search(r'^Lo[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['Lo'] = tournament_scores_df['Lo_elements'].apply(process_elements)

In [294]:
# удаляем столбец Lo_elements,тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['Lo_elements'])

In [295]:
def extract_S (title):
    # извлечение S - Сальхов (Salchow) обозначается как (S)-один из трех рёберных прыжков в фигурном катании
    pattern = r'(S\d*[<>!ex*]?[^+]*)(?=\+|$)'
    # извлекаем все S 
    jumps = re.findall(pattern, title)
    # преобразуем список в строку
    jumps_string = ', '.join(jumps)
    # удаление извлеченных комбинаций из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['S_elements', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_S(x))
)

In [296]:
def process_elements(data):
    data = data.strip()  
    if data == 'S':
        return 1
    elif re.search(r'^S[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['S'] = tournament_scores_df['S_elements'].apply(process_elements)

In [297]:
# удаляем столбец S_elements,тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['S_elements'])

In [298]:
def extract_Lz (title):
    # извлечение Lz (Lutz) – это лутц
    pattern = r'(Lz\d*[<>!ex*]?[^+]*)(?=\+|$)'
    # извлекаем все Lz 
    jumps = re.findall(pattern, title)
    # преобразуем список в строку
    jumps_string = ', '.join(jumps)
    # удаление извлеченных комбинаций из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['Lz_elements', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_Lz(x))
)

In [299]:
def process_elements(data):
    data = data.strip()  
    if data == 'Lz':
        return 1
    elif re.search(r'^Lz[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['Lz'] = tournament_scores_df['Lz_elements'].apply(process_elements)

In [300]:
# удаляем столбец Lz_elements,тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['Lz_elements'])

In [301]:
def extract_T (title):
    # извлечение T (Toeloop) – это тулуп
    pattern = r'(T\d*[<>!ex*]?[^+]*)(?=\+|$)'
    # извлекаем все T 
    jumps = re.findall(pattern, title)
    # преобразуем список в строку
    jumps_string = ', '.join(jumps)
    # удаление извлеченных комбинаций из исходной строки
    tail_title = re.sub(pattern + r'(\+|$)', '', title)
    # очистка от лишних '+'
    tail_title = re.sub(r'^\+|\+$|\+\++', '+', tail_title).strip('+')
    return jumps_string, tail_title

tournament_scores_df[['T_elements', 'tail_title']] = tournament_scores_df['tail_title'].apply(
    lambda x: pd.Series(extract_T(x))
)

In [302]:
def process_elements(data):
    data = data.strip()  
    if data == 'T':
        return 1
    elif re.search(r'^T[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['T'] = tournament_scores_df['T_elements'].apply(process_elements)

In [303]:
# удаляем столбец T_elements,тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['T_elements'])

In [304]:
# 1W Перекидной прыжок оценивается как технический элемент для уровня Пре-бронза и записывается в протоколах как "1W", базовая стоимость перекидного прыжка равна 0.2.
def process_elements(data):
    data = data.strip()  
    if data == '1W':
        return 1
    elif re.search(r'^1W[^+ ]+$', data):
        return 2
    else:
        return 0

tournament_scores_df['1W'] = tournament_scores_df['tail_title'].apply(process_elements)

In [305]:
tournament_scores_df['1W'].value_counts()

1W
0    172155
1         3
Name: count, dtype: int64

In [306]:
# удаляем столбец tail_title,тк он пуст
tournament_scores_df = tournament_scores_df.drop(columns=['tail_title'])

Я выделила с помощью регулярных выражений все возможные прыжки,вращения и тд. 1 означает,что элемент выполнен без ошибок,2 наличие ошибки и 0 элемент не выполнялся совсем

In [308]:
tournament_scores_df.to_csv('tournament_scores_df_final.csv', index=False)

In [307]:
tournament_scores_df.columns

Index(['id', 'total_score_id', 'title', 'decrease', 'base_score', 'goe',
       'avg_score', '1Lo', '2Lo', '3Lo',
       ...
       'CSp', 'SEQ', 'USpB', 'Sp', 'A', 'Lo', 'S', 'Lz', 'T', '1W'],
      dtype='object', length=103)

In [340]:
tournaments_df = pd.read_csv('data/tournaments.csv')

tournaments_df

,id,date_start,date_end,origin_id
0,1,2090-11-29,2090-12-01,2.0
1,2,2091-03-06,2091-03-10,1.0
2,3,2090-10-05,2090-10-08,2.0
3,4,2090-10-18,2090-10-21,2.0
4,5,2090-10-21,2090-10-24,2.0
...,...,...,...,...
137,7114,2092-04-15,2092-04-20,1.0
138,7116,2091-10-24,2091-10-27,0.0
139,7117,2092-04-29,2092-05-03,0.0
140,7115,2092-04-23,2092-04-26,2.0


In [341]:
units_df = pd.read_csv('data/units.csv')

units_df

,id,color,school_id
0,9474,green,244.0
1,733,green,203.0
2,734,green,235.0
3,735,green,168.0
4,736,green,168.0
...,...,...,...
4591,10122,green,198.0
4592,5108,green,62.0
4593,5109,green,27.0
4594,5110,green,62.0
